
# 참고 깃허브

https://github.com/harritaylor/torchvggish/blob/master/torchvggish/mel_features.py

https://github.com/Vidhiwar/multimodule-ecg-classification/blob/fcd8fbca95978c54b17977d219612d55ac1c30b2/ecg_cnn.py#L277

# 필요 패키지 및 함수 로드

In [2]:
import pickle as pk

In [3]:
import pickle as pk
import glob
import os
import time
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import sys
import random
from tqdm import tqdm


import gc
#from torch.utils.tensorboard import SummaryWriter
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
#from sklearn.ensemble import RandomForestClassifier


import pickle as pk

In [35]:
#로그 기록 함수 
import sys
import logging
import numpy as np
import torch
import os
import copy

def get_logger(filename):
    # Logging configuration: set the basic configuration of the logging system
    log_formatter = logging.Formatter(fmt='%(asctime)s [%(processName)s, %(process)s] [%(levelname)-5.5s]  %(message)s',
                                      datefmt='%m-%d %H:%M')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    # File logger
    file_handler = logging.FileHandler("{}.log".format(filename))
    file_handler.setFormatter(log_formatter)
    file_handler.setLevel(logging.DEBUG)
    logger.addHandler(file_handler)
    # Stderr logger
    std_handler = logging.StreamHandler(sys.stdout)
    std_handler.setFormatter(log_formatter)
    std_handler.setLevel(logging.DEBUG)
    logger.addHandler(std_handler)
    return logger


In [ ]:
#!/usr/bin/env python

# Do *not* edit this script. Changes will be discarded so that we can process the models consistently.

# This file contains functions for evaluating models for the 2022 Challenge. You can run it as follows:
#
#   python evaluate_model.py labels outputs scores.csv
#
# where 'labels' is a folder containing files with the labels, 'outputs' is a folder containing files with the outputs from your
# model, and 'scores.csv' (optional) is a collection of scores for the model outputs.
#
# Each label or output file must have the format described on the Challenge webpage. The scores for the algorithm outputs include
# the area under the receiver-operating characteristic curve (AUROC), the area under the recall-precision curve (AUPRC), macro
# accuracy, a weighted accuracy score, and the Challenge score.

import os, os.path, sys, numpy as np
from helper_code import load_patient_data, get_murmur, get_outcome, load_challenge_outputs, compare_strings

# Evaluate the models.
def evaluate_model(label_folder, output_folder):
    # Define murmur and outcome classes.
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']

    # Load and parse label and model output files.
    label_files, output_files = find_challenge_files(label_folder, output_folder)
    murmur_labels = load_murmurs(label_files, murmur_classes)
    murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
    outcome_labels = load_outcomes(label_files, outcome_classes)
    outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)

    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')

    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
        murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
        outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)

    # Return the results.
    return murmur_scores, outcome_scores

# Find Challenge files.
def find_challenge_files(label_folder, output_folder):
    label_files = list()
    output_files = list()
    for label_file in sorted(os.listdir(label_folder)):
        label_file_path = os.path.join(label_folder, label_file) # Full path for label file
        if os.path.isfile(label_file_path) and label_file.lower().endswith('.txt') and not label_file.lower().startswith('.'):
            root, ext = os.path.splitext(label_file)
            output_file = root + '.csv'
            output_file_path = os.path.join(output_folder, output_file) # Full path for corresponding output file
            if os.path.isfile(output_file_path):
                label_files.append(label_file_path)
                output_files.append(output_file_path)
            else:
                raise IOError('Output file {} not found for label file {}.'.format(output_file, label_file))

    if label_files and output_files:
        return label_files, output_files
    else:
        raise IOError('No label or output files found.')

# Load murmurs from label files.
def load_murmurs(label_files, classes):
    num_patients = len(label_files)
    num_classes = len(classes)

    # Use one-hot encoding for the labels.
    labels = np.zeros((num_patients, num_classes), dtype=np.bool_)

    # Iterate over the patients.
    for i in range(num_patients):
        data = load_patient_data(label_files[i])
        label = get_murmur(data)
        for j, x in enumerate(classes):
            if compare_strings(label, x):
                labels[i, j] = 1

    return labels

# Load outcomes from label files.
def load_outcomes(label_files, classes):
    num_patients = len(label_files)
    num_classes = len(classes)

    # Use one-hot encoding for the labels.
    labels = np.zeros((num_patients, num_classes), dtype=np.bool_)

    # Iterate over the patients.
    for i in range(num_patients):
        data = load_patient_data(label_files[i])
        label = get_outcome(data)
        for j, x in enumerate(classes):
            if compare_strings(label, x):
                labels[i, j] = 1

    return labels

# Load outputs from output files.
def load_classifier_outputs(output_files, classes):
    # The outputs should have the following form:
    #
    # #Record ID
    # class_1, class_2, class_3
    #       0,       1,       1
    #    0.12,    0.34,    0.56
    #
    num_patients = len(output_files)
    num_classes = len(classes)

    # Use one-hot encoding for the outputs.
    binary_outputs = np.zeros((num_patients, num_classes), dtype=np.bool_)
    scalar_outputs = np.zeros((num_patients, num_classes), dtype=np.float64)

    # Iterate over the patients.
    for i in range(num_patients):
        patient_id, patient_classes, patient_binary_outputs, patient_scalar_outputs = load_challenge_outputs(output_files[i])

        # Allow for unordered or reordered classes.
        for j, x in enumerate(classes):
            for k, y in enumerate(patient_classes):
                if compare_strings(x, y):
                    binary_outputs[i, j] = patient_binary_outputs[k]
                    scalar_outputs[i, j] = patient_scalar_outputs[k]

    return binary_outputs, scalar_outputs

# For each patient, set a specific class to positive if no class is positive or multiple classes are positive.
def enforce_positives(outputs, classes, positive_class):
    num_patients, num_classes = np.shape(outputs)
    j = classes.index(positive_class)

    for i in range(num_patients):
        if np.sum(outputs[i, :]) != 1:
            outputs[i, :] = 0
            outputs[i, j] = 1
    return outputs

# Compute macro AUROC and macro AUPRC.
def compute_auc(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    # Compute and summarize the confusion matrices for each class across at distinct output values.
    auroc = np.zeros(num_classes)
    auprc = np.zeros(num_classes)

    for k in range(num_classes):
        # We only need to compute TPs, FPs, FNs, and TNs at distinct output values.
        thresholds = np.unique(outputs[:, k])
        thresholds = np.append(thresholds, thresholds[-1]+1)
        thresholds = thresholds[::-1]
        num_thresholds = len(thresholds)

        # Initialize the TPs, FPs, FNs, and TNs.
        tp = np.zeros(num_thresholds)
        fp = np.zeros(num_thresholds)
        fn = np.zeros(num_thresholds)
        tn = np.zeros(num_thresholds)
        fn[0] = np.sum(labels[:, k] == 1)
        tn[0] = np.sum(labels[:, k] == 0)

        # Find the indices that result in sorted output values.
        idx = np.argsort(outputs[:, k])[::-1]

        # Compute the TPs, FPs, FNs, and TNs for class k across thresholds.
        i = 0
        for j in range(1, num_thresholds):
            # Initialize TPs, FPs, FNs, and TNs using values at previous threshold.
            tp[j] = tp[j-1]
            fp[j] = fp[j-1]
            fn[j] = fn[j-1]
            tn[j] = tn[j-1]

            # Update the TPs, FPs, FNs, and TNs at i-th output value.
            while i < num_patients and outputs[idx[i], k] >= thresholds[j]:
                if labels[idx[i], k]:
                    tp[j] += 1
                    fn[j] -= 1
                else:
                    fp[j] += 1
                    tn[j] -= 1
                i += 1

        # Summarize the TPs, FPs, FNs, and TNs for class k.
        tpr = np.zeros(num_thresholds)
        tnr = np.zeros(num_thresholds)
        ppv = np.zeros(num_thresholds)
        for j in range(num_thresholds):
            if tp[j] + fn[j]:
                tpr[j] = float(tp[j]) / float(tp[j] + fn[j])
            else:
                tpr[j] = float('nan')
            if fp[j] + tn[j]:
                tnr[j] = float(tn[j]) / float(fp[j] + tn[j])
            else:
                tnr[j] = float('nan')
            if tp[j] + fp[j]:
                ppv[j] = float(tp[j]) / float(tp[j] + fp[j])
            else:
                ppv[j] = float('nan')

        # Compute AUROC as the area under a piecewise linear function with TPR/
        # sensitivity (x-axis) and TNR/specificity (y-axis) and AUPRC as the area
        # under a piecewise constant with TPR/recall (x-axis) and PPV/precision
        # (y-axis) for class k.
        for j in range(num_thresholds-1):
            auroc[k] += 0.5 * (tpr[j+1] - tpr[j]) * (tnr[j+1] + tnr[j])
            auprc[k] += (tpr[j+1] - tpr[j]) * ppv[j+1]

    # Compute macro AUROC and macro AUPRC across classes.
    if np.any(np.isfinite(auroc)):
        macro_auroc = np.nanmean(auroc)
    else:
        macro_auroc = float('nan')
    if np.any(np.isfinite(auprc)):
        macro_auprc = np.nanmean(auprc)
    else:
        macro_auprc = float('nan')

    return macro_auroc, macro_auprc, auroc, auprc

# Compute a binary confusion matrix, where the columns are the expert labels and the rows are the classifier labels.
def compute_confusion_matrix(labels, outputs):
    assert(np.shape(labels)[0] == np.shape(outputs)[0])
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients = np.shape(labels)[0]
    num_label_classes = np.shape(labels)[1]
    num_output_classes = np.shape(outputs)[1]

    A = np.zeros((num_output_classes, num_label_classes))
    for k in range(num_patients):
        for i in range(num_output_classes):
            for j in range(num_label_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

# Compute binary one-vs-rest confusion matrices, where the columns are the expert labels and the rows are the classifier labels.
def compute_one_vs_rest_confusion_matrix(labels, outputs):
    assert(np.shape(labels) == np.shape(outputs))
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients, num_classes = np.shape(labels)

    A = np.zeros((num_classes, 2, 2))
    for i in range(num_patients):
        for j in range(num_classes):
            if labels[i, j] == 1 and outputs[i, j] == 1: # TP
                A[j, 0, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 1: # FP
                A[j, 0, 1] += 1
            elif labels[i, j] == 1 and outputs[i, j] == 0: # FN
                A[j, 1, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 0: # TN
                A[j, 1, 1] += 1

    return A

# Compute macro F-measure.
def compute_f_measure(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    A = compute_one_vs_rest_confusion_matrix(labels, outputs)

    f_measure = np.zeros(num_classes)
    for k in range(num_classes):
        tp, fp, fn, tn = A[k, 0, 0], A[k, 0, 1], A[k, 1, 0], A[k, 1, 1]
        if 2 * tp + fp + fn > 0:
            f_measure[k] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            f_measure[k] = float('nan')

    if np.any(np.isfinite(f_measure)):
        macro_f_measure = np.nanmean(f_measure)
    else:
        macro_f_measure = float('nan')

    return macro_f_measure, f_measure

# Compute accuracy.
def compute_accuracy(labels, outputs):
    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    num_patients, num_classes = np.shape(labels)
    A = compute_confusion_matrix(labels, outputs)

    # Compute accuracy.
    if np.sum(A) > 0:
        accuracy = np.trace(A) / np.sum(A)
    else:
        accuracy = float('nan')

    # Compute per-class accuracy.
    accuracy_classes = np.zeros(num_classes)
    for i in range(num_classes):
        if np.sum(A[:, i]) > 0:
            accuracy_classes[i] = A[i, i] / np.sum(A[:, i])
        else:
            accuracy_classes[i] = float('nan')

    return accuracy, accuracy_classes

# Compute accuracy.
def compute_weighted_accuracy(labels, outputs, classes):
    # Define constants.
    if classes == ['Present', 'Unknown', 'Absent']:
        weights = np.array([[5, 3, 1], [5, 3, 1], [5, 3, 1]])
    elif classes == ['Abnormal', 'Normal']:
        weights = np.array([[5, 1], [5, 1]])
    else:
        raise NotImplementedError('Weighted accuracy undefined for classes {}'.format(', '.join(classes)))

    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    A = compute_confusion_matrix(labels, outputs)

    # Multiply the confusion matrix by the weight matrix.
    assert(np.shape(A) == np.shape(weights))
    B = weights * A

    # Compute weighted_accuracy.
    if np.sum(B) > 0:
        weighted_accuracy = np.trace(B) / np.sum(B)
    else:
        weighted_accuracy = float('nan')

    return weighted_accuracy

# Define total cost for algorithmic prescreening of m patients.
def cost_algorithm(m):
    return 10*m

# Define total cost for expert screening of m patients out of a total of n total patients.
def cost_expert(m, n):
    return (25 + 397*(m/n) -1718*(m/n)**2 + 11296*(m/n)**4) * n

# Define total cost for treatment of m patients.
def cost_treatment(m):
    return 10000*m

# Define total cost for missed/late treatement of m patients.
def cost_error(m):
    return 50000*m

# Compute Challenge cost metric.
def compute_cost(labels, outputs, label_classes, output_classes):
    # Define positive and negative classes for referral and treatment.
    positive_classes = ['Present', 'Unknown', 'Abnormal']
    negative_classes = ['Absent', 'Normal']

    # Compute confusion matrix.
    A = compute_confusion_matrix(labels, outputs)

    # Identify positive and negative classes for referral.
    idx_label_positive = [i for i, x in enumerate(label_classes) if x in positive_classes]
    idx_label_negative = [i for i, x in enumerate(label_classes) if x in negative_classes]
    idx_output_positive = [i for i, x in enumerate(output_classes) if x in positive_classes]
    idx_output_negative = [i for i, x in enumerate(output_classes) if x in negative_classes]

    # Identify true positives, false positives, false negatives, and true negatives.
    tp = np.sum(A[np.ix_(idx_output_positive, idx_label_positive)])
    fp = np.sum(A[np.ix_(idx_output_positive, idx_label_negative)])
    fn = np.sum(A[np.ix_(idx_output_negative, idx_label_positive)])
    tn = np.sum(A[np.ix_(idx_output_negative, idx_label_negative)])
    total_patients = tp + fp + fn + tn

    # Compute total cost for all patients.
    total_cost = cost_algorithm(total_patients) \
        + cost_expert(tp + fp, total_patients) \
        + cost_treatment(tp) \
        + cost_error(fn)

    # Compute mean cost per patient.
    if total_patients > 0:
        mean_cost = total_cost / total_patients
    else:
        mean_cost = float('nan')

    return mean_cost



In [5]:
!nvidia-smi

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

Wed Jul 20 14:59:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   35C   

# feature extraction

In [184]:
#오디오셋으로 사전학습된 vggish 불러오기
vggish_model = torch.hub.load('harritaylor/torchvggish', 'vggish')
def vgg_feature_pretrain(fnm):
    vggish_model.eval()
    vggish_model.postprocess = False
    vggish_model.embeddings[5] = torch.nn.Sequential() # Remove last ReLU
    y,sr = load_wav_file(f)
    vggish_embeddings = vggish_model.forward(y,sr)
    vggish_embeddings =vggish_embeddings.detach().cpu().numpy()
    return vggish_embeddings.astype(np.float32)  

Using cache found in /home/hj20/.cache/torch/hub/harritaylor_torchvggish_master


In [55]:
vggish_model = torch.hub.load('harritaylor/torchvggish', 'vggish')

Using cache found in /home/hj20/.cache/torch/hub/harritaylor_torchvggish_master


In [56]:
vggish_model

VGGish(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

In [24]:
#오디오셋으로 사전학습x vggish 불러오기
def vgg_feature(fnm):
    model.train()
    model.postprocess = False
    model.embeddings[5] = torch.nn.Sequential() # Remove last ReLU
    y,sr = load_wav_file(fnm)
    vggish_embeddings = model.forward(y,sr)
    vggish_embeddings =vggish_embeddings.detach().cpu().numpy()
    return vggish_embeddings.astype(np.float32)  

In [25]:
model.embeddings[5]

ReLU(inplace=True)

In [153]:
vggish_embeddings = vggish_model.forward(y,sr).detach().cpu().numpy()

07-18 01:33 [MainProcess, 10268] [DEBUG]  bytecode dump:
>          0	NOP(arg=None, lineno=10)
           2	LOAD_GLOBAL(arg=0, lineno=10)
           4	LOAD_CONST(arg=1, lineno=10)
           6	LOAD_FAST(arg=2, lineno=10)
           8	CALL_FUNCTION(arg=2, lineno=10)
          10	STORE_FAST(arg=6, lineno=10)
          12	LOAD_CONST(arg=1, lineno=11)
          14	LOAD_FAST(arg=2, lineno=11)
          16	BINARY_TRUE_DIVIDE(arg=None, lineno=11)
          18	STORE_FAST(arg=7, lineno=11)
          20	LOAD_GLOBAL(arg=1, lineno=12)
          22	LOAD_FAST(arg=6, lineno=12)
          24	LOAD_FAST(arg=5, lineno=12)
          26	BINARY_MULTIPLY(arg=None, lineno=12)
          28	CALL_FUNCTION(arg=1, lineno=12)
          30	STORE_FAST(arg=8, lineno=12)
          32	LOAD_CONST(arg=2, lineno=13)
          34	STORE_FAST(arg=9, lineno=13)
          36	LOAD_CONST(arg=3, lineno=15)
          38	STORE_FAST(arg=10, lineno=15)
          40	LOAD_CONST(arg=2, lineno=16)
          42	STORE_FAST(arg=11, lineno=16

07-18 01:33 [MainProcess, 10268] [DEBUG]  bytecode dump:
>          0	NOP(arg=None, lineno=10)
           2	LOAD_GLOBAL(arg=0, lineno=10)
           4	LOAD_CONST(arg=1, lineno=10)
           6	LOAD_FAST(arg=2, lineno=10)
           8	CALL_FUNCTION(arg=2, lineno=10)
          10	STORE_FAST(arg=6, lineno=10)
          12	LOAD_CONST(arg=1, lineno=11)
          14	LOAD_FAST(arg=2, lineno=11)
          16	BINARY_TRUE_DIVIDE(arg=None, lineno=11)
          18	STORE_FAST(arg=7, lineno=11)
          20	LOAD_GLOBAL(arg=1, lineno=12)
          22	LOAD_FAST(arg=6, lineno=12)
          24	LOAD_FAST(arg=5, lineno=12)
          26	BINARY_MULTIPLY(arg=None, lineno=12)
          28	CALL_FUNCTION(arg=1, lineno=12)
          30	STORE_FAST(arg=8, lineno=12)
          32	LOAD_CONST(arg=2, lineno=13)
          34	STORE_FAST(arg=9, lineno=13)
          36	LOAD_CONST(arg=3, lineno=15)
          38	STORE_FAST(arg=10, lineno=15)
          40	LOAD_CONST(arg=2, lineno=16)
          42	STORE_FAST(arg=11, lineno=16

07-18 01:33 [MainProcess, 10268] [DEBUG]  bytecode dump:
>          0	NOP(arg=None, lineno=10)
           2	LOAD_GLOBAL(arg=0, lineno=10)
           4	LOAD_CONST(arg=1, lineno=10)
           6	LOAD_FAST(arg=2, lineno=10)
           8	CALL_FUNCTION(arg=2, lineno=10)
          10	STORE_FAST(arg=6, lineno=10)
          12	LOAD_CONST(arg=1, lineno=11)
          14	LOAD_FAST(arg=2, lineno=11)
          16	BINARY_TRUE_DIVIDE(arg=None, lineno=11)
          18	STORE_FAST(arg=7, lineno=11)
          20	LOAD_GLOBAL(arg=1, lineno=12)
          22	LOAD_FAST(arg=6, lineno=12)
          24	LOAD_FAST(arg=5, lineno=12)
          26	BINARY_MULTIPLY(arg=None, lineno=12)
          28	CALL_FUNCTION(arg=1, lineno=12)
          30	STORE_FAST(arg=8, lineno=12)
          32	LOAD_CONST(arg=2, lineno=13)
          34	STORE_FAST(arg=9, lineno=13)
          36	LOAD_CONST(arg=3, lineno=15)
          38	STORE_FAST(arg=10, lineno=15)
          40	LOAD_CONST(arg=2, lineno=16)
          42	STORE_FAST(arg=11, lineno=16

07-18 01:33 [MainProcess, 10268] [DEBUG]  bytecode dump:
>          0	NOP(arg=None, lineno=10)
           2	LOAD_GLOBAL(arg=0, lineno=10)
           4	LOAD_CONST(arg=1, lineno=10)
           6	LOAD_FAST(arg=2, lineno=10)
           8	CALL_FUNCTION(arg=2, lineno=10)
          10	STORE_FAST(arg=6, lineno=10)
          12	LOAD_CONST(arg=1, lineno=11)
          14	LOAD_FAST(arg=2, lineno=11)
          16	BINARY_TRUE_DIVIDE(arg=None, lineno=11)
          18	STORE_FAST(arg=7, lineno=11)
          20	LOAD_GLOBAL(arg=1, lineno=12)
          22	LOAD_FAST(arg=6, lineno=12)
          24	LOAD_FAST(arg=5, lineno=12)
          26	BINARY_MULTIPLY(arg=None, lineno=12)
          28	CALL_FUNCTION(arg=1, lineno=12)
          30	STORE_FAST(arg=8, lineno=12)
          32	LOAD_CONST(arg=2, lineno=13)
          34	STORE_FAST(arg=9, lineno=13)
          36	LOAD_CONST(arg=3, lineno=15)
          38	STORE_FAST(arg=10, lineno=15)
          40	LOAD_CONST(arg=2, lineno=16)
          42	STORE_FAST(arg=11, lineno=16

07-18 01:33 [MainProcess, 10268] [DEBUG]  bytecode dump:
>          0	NOP(arg=None, lineno=10)
           2	LOAD_GLOBAL(arg=0, lineno=10)
           4	LOAD_CONST(arg=1, lineno=10)
           6	LOAD_FAST(arg=2, lineno=10)
           8	CALL_FUNCTION(arg=2, lineno=10)
          10	STORE_FAST(arg=6, lineno=10)
          12	LOAD_CONST(arg=1, lineno=11)
          14	LOAD_FAST(arg=2, lineno=11)
          16	BINARY_TRUE_DIVIDE(arg=None, lineno=11)
          18	STORE_FAST(arg=7, lineno=11)
          20	LOAD_GLOBAL(arg=1, lineno=12)
          22	LOAD_FAST(arg=6, lineno=12)
          24	LOAD_FAST(arg=5, lineno=12)
          26	BINARY_MULTIPLY(arg=None, lineno=12)
          28	CALL_FUNCTION(arg=1, lineno=12)
          30	STORE_FAST(arg=8, lineno=12)
          32	LOAD_CONST(arg=2, lineno=13)
          34	STORE_FAST(arg=9, lineno=13)
          36	LOAD_CONST(arg=3, lineno=15)
          38	STORE_FAST(arg=10, lineno=15)
          40	LOAD_CONST(arg=2, lineno=16)
          42	STORE_FAST(arg=11, lineno=16

...

07-18 01:33 [MainProcess, 10268] [DEBUG]  block_infos State(pc_initial=466 nstack_initial=1):
AdaptBlockInfo(insts=((466, {'res': '$time_register466.1'}), (468, {'res': '$time_increment468.2'}), (470, {'lhs': '$time_register466.1', 'rhs': '$time_increment468.2', 'res': '$470inplace_add.3'}), (472, {'value': '$470inplace_add.3'}), (474, {})), outgoing_phis={}, blockstack=({'kind': BlockKind(LOOP), 'end': 478, 'entry_stack': 0, 'stack_depth': 0, 'handler': None},), active_try_block=None, outgoing_edgepushed={112: ('$phi466.0',)})
07-18 01:33 [MainProcess, 10268] [DEBUG]  block_infos State(pc_initial=466 nstack_initial=1):
AdaptBlockInfo(insts=((466, {'res': '$time_register466.1'}), (468, {'res': '$time_increment468.2'}), (470, {'lhs': '$time_register466.1', 'rhs': '$time_increment468.2', 'res': '$470inplace_add.3'}), (472, {'value': '$470inplace_add.3'}), (474, {})), outgoing_phis={}, blockstack=({'kind': BlockKind(LOOP), 'end': 478, 'entry_stack': 0, 'stack_depth': 0, 'handler': None},)

07-18 01:33 [MainProcess, 10268] [DEBUG]  label 0:
    x = arg(0, name=x)                       ['x']
    y = arg(1, name=y)                       ['y']
    sample_ratio = arg(2, name=sample_ratio) ['sample_ratio']
    interp_win = arg(3, name=interp_win)     ['interp_win']
    interp_delta = arg(4, name=interp_delta) ['interp_delta']
    num_table = arg(5, name=num_table)       ['num_table']
    $2load_global.0 = global(min: <built-in function min>) ['$2load_global.0']
    $const4.1 = const(float, 1.0)            ['$const4.1']
    $8call_function.3 = call $2load_global.0($const4.1, sample_ratio, func=$2load_global.0, args=[Var($const4.1, interpn.py:10), Var(sample_ratio, interpn.py:10)], kws=(), vararg=None) ['$2load_global.0', '$8call_function.3', '$const4.1', 'sample_ratio']
    scale = $8call_function.3                ['$8call_function.3', 'scale']
    $const12.4 = const(float, 1.0)           ['$const12.4']
    $16binary_true_divide.6 = $const12.4 / sample_ratio ['$16binary_true_di

07-18 01:33 [MainProcess, 10268] [DEBUG]  label 0:
    x = arg(0, name=x)                       ['x']
    y = arg(1, name=y)                       ['y']
    sample_ratio = arg(2, name=sample_ratio) ['sample_ratio']
    interp_win = arg(3, name=interp_win)     ['interp_win']
    interp_delta = arg(4, name=interp_delta) ['interp_delta']
    num_table = arg(5, name=num_table)       ['num_table']
    $2load_global.0 = global(min: <built-in function min>) ['$2load_global.0']
    $const4.1 = const(float, 1.0)            ['$const4.1']
    $8call_function.3 = call $2load_global.0($const4.1, sample_ratio, func=$2load_global.0, args=[Var($const4.1, interpn.py:10), Var(sample_ratio, interpn.py:10)], kws=(), vararg=None) ['$2load_global.0', '$8call_function.3', '$const4.1', 'sample_ratio']
    scale = $8call_function.3                ['$8call_function.3', 'scale']
    $const12.4 = const(float, 1.0)           ['$const12.4']
    $16binary_true_divide.6 = $const12.4 / sample_ratio ['$16binary_true_di

07-18 01:33 [MainProcess, 10268] [DEBUG]  label 0:
    x = arg(0, name=x)                       ['x']
    y = arg(1, name=y)                       ['y']
    sample_ratio = arg(2, name=sample_ratio) ['sample_ratio']
    interp_win = arg(3, name=interp_win)     ['interp_win']
    interp_delta = arg(4, name=interp_delta) ['interp_delta']
    num_table = arg(5, name=num_table)       ['num_table']
    $2load_global.0 = global(min: <built-in function min>) ['$2load_global.0']
    $const4.1 = const(float, 1.0)            ['$const4.1']
    $8call_function.3 = call $2load_global.0($const4.1, sample_ratio, func=$2load_global.0, args=[Var($const4.1, interpn.py:10), Var(sample_ratio, interpn.py:10)], kws=(), vararg=None) ['$2load_global.0', '$8call_function.3', '$const4.1', 'sample_ratio']
    scale = $8call_function.3                ['$8call_function.3', 'scale']
    $const12.4 = const(float, 1.0)           ['$const12.4']
    $16binary_true_divide.6 = $const12.4 / sample_ratio ['$16binary_true_di

07-18 01:33 [MainProcess, 10268] [DEBUG]  label 0:
    x = arg(0, name=x)                       ['x']
    y = arg(1, name=y)                       ['y']
    sample_ratio = arg(2, name=sample_ratio) ['sample_ratio']
    interp_win = arg(3, name=interp_win)     ['interp_win']
    interp_delta = arg(4, name=interp_delta) ['interp_delta']
    num_table = arg(5, name=num_table)       ['num_table']
    $2load_global.0 = global(min: <built-in function min>) ['$2load_global.0']
    $const4.1 = const(float, 1.0)            ['$const4.1']
    $8call_function.3 = call $2load_global.0($const4.1, sample_ratio, func=$2load_global.0, args=[Var($const4.1, interpn.py:10), Var(sample_ratio, interpn.py:10)], kws=(), vararg=None) ['$2load_global.0', '$8call_function.3', '$const4.1', 'sample_ratio']
    scale = $8call_function.3                ['$8call_function.3', 'scale']
    $const12.4 = const(float, 1.0)           ['$const12.4']
    $16binary_true_divide.6 = $const12.4 / sample_ratio ['$16binary_true_di

In [38]:
for i in tqdm(range(num_patient_files)):

# Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    num_locations = get_num_locations(current_patient_data)
    recording_information = current_patient_data.split('\n')[1:num_locations+1]
    for j in range(num_locations) :
        entries = recording_information[j].split(' ')
        recording_file = entries[2]
        filename = os.path.join(data_folder, recording_file)


100%|██████████| 751/751 [00:00<00:00, 41542.55it/s]


In [27]:
from tqdm import tqdm

In [39]:
filename

'/home/hj20/hmd/murmur/train/85349_TV.wav'

In [26]:
def get_features_vgg(data_folder, patient_files_trn) :
    features = []
    murmurs=[] 
    outcomes=[]

    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in tqdm(range(num_patient_files)):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract melspec
            
            mel1 = vgg_feature(filename)
            features.append(mel1)


            # Extract labels and use one-hot encoding.
            current_murmur = np.zeros(num_murmur_classes, dtype=int)
            murmur = get_murmur(current_patient_data)
            if murmur in murmur_classes:
                j = murmur_classes.index(murmur)
                current_murmur[j] = 1
            murmurs.append(current_murmur)
            
            
            current_outcome = np.zeros(num_outcome_classes, dtype=int)
            outcome = get_outcome(current_patient_data)
            if outcome in outcome_classes:
                j = outcome_classes.index(outcome)
                current_outcome[j] = 1
            outcomes.append(current_outcome)
            
            
            
            
            
    #M, N = features['mel1'][i].shape
    #for i in range(len(features['mel1'])) :
    #    features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    murmurs = np.array(murmurs)
    outcomes = np.array(outcomes)
    
    #or k1 in features.keys() :
    #   features[k1] = np.array(features[k1])
    
    return features, murmurs, outcomes

In [65]:
murmurs

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [6]:
!pip install jupyter_contrib_nbextensions

^C
Traceback (most recent call last):
  File "/home/hj20/.local/bin/pip", line 11, in <module>
    sys.exit(main())
  File "/home/hj20/.local/lib/python2.7/site-packages/pip/_internal/cli/main.py", line 73, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/home/hj20/.local/lib/python2.7/site-packages/pip/_internal/commands/__init__.py", line 105, in create_command
    module = importlib.import_module(module_path)
  File "/usr/lib/python2.7/importlib/__init__.py", line 37, in import_module
    __import__(name)
  File "/home/hj20/.local/lib/python2.7/site-packages/pip/_internal/commands/install.py", line 17, in <module>
    from pip._internal.cli.req_command import RequirementCommand, with_cleanup
  File "/home/hj20/.local/lib/python2.7/site-packages/pip/_internal/cli/req_command.py", line 21, in <module>
    from pip._internal.network.session import PipSession
  File "/home/hj20/.local/lib/python2.7/site-packages/pip/_internal/network/session.p

In [ ]:
! jupyter contrib nbextension install --user

In [4]:
!ls

 Dockerfile	  __pycache__	     run_model.py    'VGGish 임베딩_Hj.ipynb'
 helper_code.py   README.md	     team_code.py     vggish.ipynb
 LICENSE	  requirements.txt   train_model.py


In [7]:
data_folder = "/home/hj20/hmd/murmur/train/"
model_folder = "/home/hj20/hmd/python-classifier-2022/model"

In [8]:
!pwd

/home/hj20/hmd/python-classifier-2022


In [9]:
patient_files = find_patient_files(data_folder)

In [10]:
num_patient_files = len(patient_files)

In [11]:
num_patient_files

751

In [12]:
patient_files

['/home/hj20/hmd/murmur/train/2530.txt',
 '/home/hj20/hmd/murmur/train/9979.txt',
 '/home/hj20/hmd/murmur/train/9983.txt',
 '/home/hj20/hmd/murmur/train/13918.txt',
 '/home/hj20/hmd/murmur/train/14241.txt',
 '/home/hj20/hmd/murmur/train/14998.txt',
 '/home/hj20/hmd/murmur/train/23625.txt',
 '/home/hj20/hmd/murmur/train/24160.txt',
 '/home/hj20/hmd/murmur/train/29045.txt',
 '/home/hj20/hmd/murmur/train/29378.txt',
 '/home/hj20/hmd/murmur/train/31737.txt',
 '/home/hj20/hmd/murmur/train/36327.txt',
 '/home/hj20/hmd/murmur/train/39043.txt',
 '/home/hj20/hmd/murmur/train/39403.txt',
 '/home/hj20/hmd/murmur/train/39456.txt',
 '/home/hj20/hmd/murmur/train/40058.txt',
 '/home/hj20/hmd/murmur/train/40840.txt',
 '/home/hj20/hmd/murmur/train/43852.txt',
 '/home/hj20/hmd/murmur/train/44514.txt',
 '/home/hj20/hmd/murmur/train/45843.txt',
 '/home/hj20/hmd/murmur/train/46065.txt',
 '/home/hj20/hmd/murmur/train/46532.txt',
 '/home/hj20/hmd/murmur/train/46579.txt',
 '/home/hj20/hmd/murmur/train/46778.t

## vgg 피쳐

In [40]:
y,sr = load_wav_file(filename)

In [151]:
sr #sampling rate

4000

In [42]:
y.shape

(78592,)

In [13]:
def periodic_hann(window_length):
  """Calculate a "periodic" Hann window.
  The classic Hann window is defined as a raised cosine that starts and
  ends on zero, and where every value appears twice, except the middle
  point for an odd-length window.  Matlab calls this a "symmetric" window
  and np.hanning() returns it.  However, for Fourier analysis, this
  actually represents just over one cycle of a period N-1 cosine, and
  thus is not compactly expressed on a length-N Fourier basis.  Instead,
  it's better to use a raised cosine that ends just before the final
  zero value - i.e. a complete cycle of a period-N cosine.  Matlab
  calls this a "periodic" window. This routine calculates it.
  Args:
    window_length: The number of points in the returned window.
  Returns:
    A 1D np.array containing the periodic hann window.
  """
  return 0.5 - (0.5 * np.cos(2 * np.pi / window_length *
                             np.arange(window_length)))

In [14]:
def stft_magnitude(signal, fft_length,
                   hop_length=None,
                   window_length=None):
    """Calculate the short-time Fourier transform magnitude.
    Args:
    signal: 1D np.array of the input time-domain signal.
    fft_length: Size of the FFT to apply.
    hop_length: Advance (in samples) between each frame passed to FFT.
    window_length: Length of each block of samples to pass to FFT.
    Returns:
    2D np.array where each row contains the magnitudes of the fft_length/2+1
    unique values of the FFT for the corresponding frame of input samples.
    """
    frames = frame(signal, window_length, hop_length)
    # Apply frame window to each frame. We use a periodic Hann (cosine of period
    # window_length) instead of the symmetric Hann of np.hanning (period
    # window_length-1).
    window = periodic_hann(window_length)
    windowed_frames = frames * window
    return np.abs(np.fft.rfft(windowed_frames, int(fft_length)))

In [15]:
"""Convert array into a sequence of successive possibly overlapping frames.
      An n-dimensional array of shape (num_samples, ...) is converted into an
      (n+1)-D array of shape (num_frames, window_length, ...), where each frame
      starts hop_length points after the preceding one.
      This is accomplished using stride_tricks, so the original data is not
      copied.  However, there is no zero-padding, so any incomplete frames at the
      end are not included.
      Args:
        data: np.array of dimension N >= 1.
        window_length: Number of samples in each frame.
        hop_length: Advance (in samples) between each window.
      Returns:
        (N+1)-D np.array with as many rows as there are complete frames that can be
        extracted.
"""
def frame(data, window_length, hop_length):
    num_samples = data.shape[0]
    num_frames = 1 + int(np.floor((num_samples - window_length) / hop_length))
    shape = (num_frames, window_length) + data.shape[1:]
    strides = (data.strides[0] * hop_length,) + data.strides
    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

In [16]:
def spectrogram_to_mel_matrix(num_mel_bins=20,
                              num_spectrogram_bins=129,
                              audio_sample_rate=4000,
                              lower_edge_hertz=125.0,
                              upper_edge_hertz=3800.0):
 
    nyquist_hertz = audio_sample_rate / 2.
    if lower_edge_hertz < 0.0:
        raise ValueError("lower_edge_hertz %.1f must be >= 0" % lower_edge_hertz)
    if lower_edge_hertz >= upper_edge_hertz:
        raise ValueError("lower_edge_hertz %.1f >= upper_edge_hertz %.1f" %
                     (lower_edge_hertz, upper_edge_hertz))
    if upper_edge_hertz > nyquist_hertz:
        raise ValueError("upper_edge_hertz %.1f is greater than Nyquist %.1f" %
                     (upper_edge_hertz, nyquist_hertz))
    spectrogram_bins_hertz = np.linspace(0.0, nyquist_hertz, num_spectrogram_bins)
    spectrogram_bins_mel = hertz_to_mel(spectrogram_bins_hertz)
    # The i'th mel band (starting from i=1) has center frequency
    # band_edges_mel[i], lower edge band_edges_mel[i-1], and higher edge
    # band_edges_mel[i+1].  Thus, we need num_mel_bins + 2 values in
    # the band_edges_mel arrays.
    band_edges_mel = np.linspace(hertz_to_mel(lower_edge_hertz),
                               hertz_to_mel(upper_edge_hertz), num_mel_bins + 2)
    # Matrix to post-multiply feature arrays whose rows are num_spectrogram_bins
    # of spectrogram values.
    mel_weights_matrix = np.empty((num_spectrogram_bins, num_mel_bins))
    for i in range(num_mel_bins):
        lower_edge_mel, center_mel, upper_edge_mel = band_edges_mel[i:i + 3]
        # Calculate lower and upper slopes for every spectrogram bin.
        # Line segments are linear in the *mel* domain, not hertz.
        lower_slope = ((spectrogram_bins_mel - lower_edge_mel) /
                   (center_mel - lower_edge_mel))
        upper_slope = ((upper_edge_mel - spectrogram_bins_mel) /
                   (upper_edge_mel - center_mel))
    # .. then intersect them with each other and zero.
        mel_weights_matrix[:, i] = np.maximum(0.0, np.minimum(lower_slope,
                                                          upper_slope))
  # HTK excludes the spectrogram DC bin; make sure it always gets a zero
  # coefficient.
    mel_weights_matrix[0, :] = 0.0
    return mel_weights_matrix

In [17]:
def log_mel_spectrogram(data,
                        audio_sample_rate=4000,
                        log_offset=0.0,
                        window_length_secs=0.025,
                        hop_length_secs=0.010,
                        **kwargs):
    """Convert waveform to a log magnitude mel-frequency spectrogram.
    Args:
    data: 1D np.array of waveform data.
    audio_sample_rate: The sampling rate of data.
    log_offset: Add this to values when taking log to avoid -Infs.
    window_length_secs: Duration of each window to analyze.
    hop_length_secs: Advance between successive analysis windows.
    **kwargs: Additional arguments to pass to spectrogram_to_mel_matrix.
    Returns:
    2D np.array of (num_frames, num_mel_bins) consisting of log mel filterbank
    magnitudes for successive frames.
    """
    window_length_samples = int(round(audio_sample_rate * window_length_secs))
    hop_length_samples = int(round(audio_sample_rate * hop_length_secs))
    fft_length = 2 ** int(np.ceil(np.log(window_length_samples) / np.log(2.0)))
    spectrogram = stft_magnitude(
      data,
      fft_length=fft_length,
      hop_length=hop_length_samples,
      window_length=window_length_samples)
    mel_spectrogram = np.dot(spectrogram, spectrogram_to_mel_matrix(
      num_spectrogram_bins=spectrogram.shape[1],
      audio_sample_rate=audio_sample_rate, **kwargs))
    return np.log(mel_spectrogram + log_offset)

In [18]:
import torch

import numpy as np
import resampy

import soundfile as sf


def waveform_to_examples(data, sample_rate, return_tensor=True):
    """Converts audio waveform into an array of examples for VGGish.
  Args:
    data: np.array of either one dimension (mono) or two dimensions
      (multi-channel, with the outer dimension representing channels).
      Each sample is generally expected to lie in the range [-1.0, +1.0],
      although this is not required.
    sample_rate: Sample rate of data.
    return_tensor: Return data as a Pytorch tensor ready for VGGish
  Returns:
    3-D np.array of shape [num_examples, num_frames, num_bands] which represents
    a sequence of examples, each of which contains a patch of log mel
    spectrogram, covering num_frames frames of audio and num_bands mel frequency
    bands, where the frame length is vggish_params.STFT_HOP_LENGTH_SECONDS.
  """
    # Convert to mono.
    if len(data.shape) > 1:
        data = np.mean(data, axis=1)
    # Resample to the rate assumed by VGGish.
    if sample_rate != SAMPLE_RATE:
        data = resampy.resample(data, sample_rate, SAMPLE_RATE)

    # Compute log mel spectrogram features.
    log_mel = log_mel_spectrogram(
        data,
        audio_sample_rate=SAMPLE_RATE,
        log_offset=LOG_OFFSET,
        window_length_secs=STFT_WINDOW_LENGTH_SECONDS,
        hop_length_secs=STFT_HOP_LENGTH_SECONDS,
        num_mel_bins=NUM_MEL_BINS,
        lower_edge_hertz=MEL_MIN_HZ,
        upper_edge_hertz=MEL_MAX_HZ)

    # Frame features into examples.
    features_sample_rate = 1.0 / STFT_HOP_LENGTH_SECONDS
    example_window_length = int(round(
        EXAMPLE_WINDOW_SECONDS * features_sample_rate))
    example_hop_length = int(round(
        EXAMPLE_HOP_SECONDS * features_sample_rate))
    log_mel_examples = frame(
        log_mel,
        window_length=example_window_length,
        hop_length=example_hop_length)

    if return_tensor:
        log_mel_examples = torch.tensor(
            log_mel_examples, requires_grad=True)[:, None, :, :].float()

    return log_mel_examples


def wavfile_to_examples(wav_file, return_tensor=True):
    """Convenience wrapper around waveform_to_examples() for a common WAV format.
  Args:
    wav_file: String path to a file, or a file-like object. The file
    is assumed to contain WAV audio data with signed 16-bit PCM samples.
    torch: Return data as a Pytorch tensor ready for VGGish
  Returns:
    See waveform_to_examples.
  """
    wav_data, sr = sf.read(wav_file, dtype='int16')
    assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
    samples = wav_data / 32768.0  # Convert to [-1.0, +1.0]
    return waveform_to_examples(samples, sr, return_tensor)

In [19]:
# Architectural constants.
NUM_FRAMES = 96  # Frames in input mel-spectrogram patch.
NUM_BANDS = 64  # Frequency bands in input mel-spectrogram patch.
EMBEDDING_SIZE = 128  # Size of embedding layer.

# Hyperparameters used in feature and example generation.
SAMPLE_RATE = 4000
STFT_WINDOW_LENGTH_SECONDS = 0.025
STFT_HOP_LENGTH_SECONDS = 0.010
NUM_MEL_BINS = NUM_BANDS
MEL_MIN_HZ = 125
MEL_MAX_HZ = 2000
LOG_OFFSET = 0.01  # Offset used for stabilized log of input mel-spectrogram.
EXAMPLE_WINDOW_SECONDS = 0.96  # Each example contains 96 10ms frames
EXAMPLE_HOP_SECONDS = 0.96  # with zero overlap.

# Parameters used for embedding postprocessing.
PCA_EIGEN_VECTORS_NAME = 'pca_eigen_vectors'
PCA_MEANS_NAME = 'pca_means'
QUANTIZE_MIN_VAL = -2.0
QUANTIZE_MAX_VAL = +2.0

# Hyperparameters used in training.
INIT_STDDEV = 0.01  # Standard deviation used to initialize weights.
LEARNING_RATE = 1e-4  # Learning rate for the Adam optimizer.
ADAM_EPSILON = 1e-8  # Epsilon for the Adam optimizer.

# Names of ops, tensors, and features.
INPUT_OP_NAME = 'vggish/input_features'
INPUT_TENSOR_NAME = INPUT_OP_NAME + ':0'
OUTPUT_OP_NAME = 'vggish/embedding'
OUTPUT_TENSOR_NAME = OUTPUT_OP_NAME + ':0'
AUDIO_EMBEDDING_FEATURE_NAME = 'audio_embedding'

In [20]:
# Mel spectrum constants and functions.
_MEL_BREAK_FREQUENCY_HERTZ = 700.0
_MEL_HIGH_FREQUENCY_Q = 1127.0

In [21]:
def hertz_to_mel(frequencies_hertz):
  """Convert frequencies to mel scale using HTK formula.
  Args:
    frequencies_hertz: Scalar or np.array of frequencies in hertz.
  Returns:
    Object of same size as frequencies_hertz containing corresponding values
    on the mel scale.
  """
  return _MEL_HIGH_FREQUENCY_Q * np.log(
      1.0 + (frequencies_hertz / _MEL_BREAK_FREQUENCY_HERTZ))


def spectrogram_to_mel_matrix(num_mel_bins=20,
                              num_spectrogram_bins=129,
                              audio_sample_rate=8000,
                              lower_edge_hertz=125.0,
                              upper_edge_hertz=3800.0):
  """Return a matrix that can post-multiply spectrogram rows to make mel.
  Returns a np.array matrix A that can be used to post-multiply a matrix S of
  spectrogram values (STFT magnitudes) arranged as frames x bins to generate a
  "mel spectrogram" M of frames x num_mel_bins.  M = S A.
  The classic HTK algorithm exploits the complementarity of adjacent mel bands
  to multiply each FFT bin by only one mel weight, then add it, with positive
  and negative signs, to the two adjacent mel bands to which that bin
  contributes.  Here, by expressing this operation as a matrix multiply, we go
  from num_fft multiplies per frame (plus around 2*num_fft adds) to around
  num_fft^2 multiplies and adds.  However, because these are all presumably
  accomplished in a single call to np.dot(), it's not clear which approach is
  faster in Python.  The matrix multiplication has the attraction of being more
  general and flexible, and much easier to read.
  Args:
    num_mel_bins: How many bands in the resulting mel spectrum.  This is
      the number of columns in the output matrix.
    num_spectrogram_bins: How many bins there are in the source spectrogram
      data, which is understood to be fft_size/2 + 1, i.e. the spectrogram
      only contains the nonredundant FFT bins.
    audio_sample_rate: Samples per second of the audio at the input to the
      spectrogram. We need this to figure out the actual frequencies for
      each spectrogram bin, which dictates how they are mapped into mel.
    lower_edge_hertz: Lower bound on the frequencies to be included in the mel
      spectrum.  This corresponds to the lower edge of the lowest triangular
      band.
    upper_edge_hertz: The desired top edge of the highest frequency band.
  Returns:
    An np.array with shape (num_spectrogram_bins, num_mel_bins).
  Raises:
    ValueError: if frequency edges are incorrectly ordered or out of range.
  """
  nyquist_hertz = audio_sample_rate / 2.
  if lower_edge_hertz < 0.0:
    raise ValueError("lower_edge_hertz %.1f must be >= 0" % lower_edge_hertz)
  if lower_edge_hertz >= upper_edge_hertz:
    raise ValueError("lower_edge_hertz %.1f >= upper_edge_hertz %.1f" %
                     (lower_edge_hertz, upper_edge_hertz))
  if upper_edge_hertz > nyquist_hertz:
    raise ValueError("upper_edge_hertz %.1f is greater than Nyquist %.1f" %
                     (upper_edge_hertz, nyquist_hertz))
  spectrogram_bins_hertz = np.linspace(0.0, nyquist_hertz, num_spectrogram_bins)
  spectrogram_bins_mel = hertz_to_mel(spectrogram_bins_hertz)
  # The i'th mel band (starting from i=1) has center frequency
  # band_edges_mel[i], lower edge band_edges_mel[i-1], and higher edge
  # band_edges_mel[i+1].  Thus, we need num_mel_bins + 2 values in
  # the band_edges_mel arrays.
  band_edges_mel = np.linspace(hertz_to_mel(lower_edge_hertz),
                               hertz_to_mel(upper_edge_hertz), num_mel_bins + 2)
  # Matrix to post-multiply feature arrays whose rows are num_spectrogram_bins
  # of spectrogram values.
  mel_weights_matrix = np.empty((num_spectrogram_bins, num_mel_bins))
  for i in range(num_mel_bins):
    lower_edge_mel, center_mel, upper_edge_mel = band_edges_mel[i:i + 3]
    # Calculate lower and upper slopes for every spectrogram bin.
    # Line segments are linear in the *mel* domain, not hertz.
    lower_slope = ((spectrogram_bins_mel - lower_edge_mel) /
                   (center_mel - lower_edge_mel))
    upper_slope = ((upper_edge_mel - spectrogram_bins_mel) /
                   (upper_edge_mel - center_mel))
    # .. then intersect them with each other and zero.
    mel_weights_matrix[:, i] = np.maximum(0.0, np.minimum(lower_slope,
                                                          upper_slope))
  # HTK excludes the spectrogram DC bin; make sure it always gets a zero
  # coefficient.
  mel_weights_matrix[0, :] = 0.0
  return mel_weights_matrix


In [22]:
import numpy as np
import torch
import torch.nn as nn
from torch import hub



class VGG(nn.Module):
    def __init__(self, features):
        super(VGG, self).__init__()
        self.features = features
        self.embeddings = nn.Sequential(
            nn.Linear(512 * 4 * 6, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 128),
            nn.ReLU(True))

    def forward(self, x):
        x = self.features(x)

        # Transpose the output from features to
        # remain compatible with vggish embeddings
        x = torch.transpose(x, 1, 3)
        x = torch.transpose(x, 1, 2)
        x = x.contiguous()
        x = x.view(x.size(0), -1)

        return self.embeddings(x)


class Postprocessor(nn.Module):
    """Post-processes VGGish embeddings. Returns a torch.Tensor instead of a
    numpy array in order to preserve the gradient.
    "The initial release of AudioSet included 128-D VGGish embeddings for each
    segment of AudioSet. These released embeddings were produced by applying
    a PCA transformation (technically, a whitening transform is included as well)
    and 8-bit quantization to the raw embedding output from VGGish, in order to
    stay compatible with the YouTube-8M project which provides visual embeddings
    in the same format for a large set of YouTube videos. This class implements
    the same PCA (with whitening) and quantization transformations."
    """

    def __init__(self):
        """Constructs a postprocessor."""
        super(Postprocessor, self).__init__()
        # Create empty matrix, for user's state_dict to load
        self.pca_eigen_vectors = torch.empty(
            (EMBEDDING_SIZE, EMBEDDING_SIZE,),
            dtype=torch.float,
        )
        self.pca_means = torch.empty(
            (EMBEDDING_SIZE, 1), dtype=torch.float
        )

        self.pca_eigen_vectors = nn.Parameter(self.pca_eigen_vectors, requires_grad=False)
        self.pca_means = nn.Parameter(self.pca_means, requires_grad=False)

    def postprocess(self, embeddings_batch):
        """Applies tensor postprocessing to a batch of embeddings.
        Args:
          embeddings_batch: An tensor of shape [batch_size, embedding_size]
            containing output from the embedding layer of VGGish.
        Returns:
          A tensor of the same shape as the input, containing the PCA-transformed,
          quantized, and clipped version of the input.
        """
        assert len(embeddings_batch.shape) == 2, "Expected 2-d batch, got %r" % (
            embeddings_batch.shape,
        )
        assert (
            embeddings_batch.shape[1] == EMBEDDING_SIZE
        ), "Bad batch shape: %r" % (embeddings_batch.shape,)

        # Apply PCA.
        # - Embeddings come in as [batch_size, embedding_size].
        # - Transpose to [embedding_size, batch_size].
        # - Subtract pca_means column vector from each column.
        # - Premultiply by PCA matrix of shape [output_dims, input_dims]
        #   where both are are equal to embedding_size in our case.
        # - Transpose result back to [batch_size, embedding_size].
        pca_applied = torch.mm(self.pca_eigen_vectors, (embeddings_batch.t() - self.pca_means)).t()

        # Quantize by:
        # - clipping to [min, max] range
        clipped_embeddings = torch.clamp(
            pca_applied, QUANTIZE_MIN_VAL,QUANTIZE_MAX_VAL
        )
        # - convert to 8-bit in range [0.0, 255.0]
        quantized_embeddings = torch.round(
            (clipped_embeddings - QUANTIZE_MIN_VAL)
            * (
                255.0
                / (QUANTIZE_MAX_VAL -QUANTIZE_MIN_VAL)
            )
        )
        return torch.squeeze(quantized_embeddings)

    def forward(self, x):
        return self.postprocess(x)


def make_layers():
    layers = []
    in_channels = 1
    for v in [64, "M", 128, "M", 256, 256, "M", 512, 512, "M"]:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


def _vgg():
    return VGG(make_layers())


# def _spectrogram():
#     config = dict(
#         sr=16000,
#         n_fft=400,
#         n_mels=64,
#         hop_length=160,
#         window="hann",
#         center=False,
#         pad_mode="reflect",
#         htk=True,
#         fmin=125,
#         fmax=7500,
#         output_format='Magnitude',
#         #             device=device,
#     )
#     return Spectrogram.MelSpectrogram(**config)


class VGGish(VGG):
    def __init__(self, device=None, pretrained=False, preprocess=True, postprocess=False, progress=True):
        super().__init__(make_layers())
        if pretrained:
            state_dict = hub.load_state_dict_from_url(urls['vggish'], progress=progress)
            super().load_state_dict(state_dict)

        if device is None:
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.device = device
        self.preprocess = preprocess
        self.postprocess = postprocess
        if self.postprocess:
            self.pproc = Postprocessor()
            if pretrained:
                state_dict = hub.load_state_dict_from_url(urls['pca'], progress=progress)
                # TODO: Convert the state_dict to torch
                state_dict[PCA_EIGEN_VECTORS_NAME] = torch.as_tensor(
                    state_dict[PCA_EIGEN_VECTORS_NAME], dtype=torch.float
                )
                state_dict[PCA_MEANS_NAME] = torch.as_tensor(
                    state_dict[PCA_MEANS_NAME].reshape(-1, 1), dtype=torch.float
                )

                self.pproc.load_state_dict(state_dict)
        self.to(self.device)

    def forward(self, x, fs=None):
        if self.preprocess:
            x = self._preprocess(x, fs)
        x = x.to(self.device)
        x = VGG.forward(self, x)
        if self.postprocess:
            x = self._postprocess(x)
        return x

    def _preprocess(self, x, fs):
        if isinstance(x, np.ndarray):
            x = waveform_to_examples(x, fs)
        elif isinstance(x, str):
            x = wavfile_to_examples(x)
        else:
            raise AttributeError
        return x

    def _postprocess(self, x):
        return self.pproc(x)

In [23]:
model = VGGish()

In [87]:
model(y,sr).shape

torch.Size([20, 128])

In [66]:
data = resampy.resample(y, sample_rate, SAMPLE_RATE)

In [65]:
sample_rate = 4000

In [48]:
!nvidia-smi

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

Wed Jul 20 13:18:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   41C   

In [24]:
# Load a WAV file.
def load_wav_file(filename):
    frequency, recording = sp.io.wavfile.read(filename)
    return recording, frequency

In [25]:
import os, numpy as np, scipy as sp, scipy.io, scipy.io.wavfile

In [13]:
num_locations

NameError: name 'num_locations' is not defined

In [101]:
num_patient_files

751

In [113]:
len(features)

2532

In [114]:
2532/4

633.0

In [222]:
patient_files

['/home/hj20/hmd/murmur/test/33151.txt',
 '/home/hj20/hmd/murmur/test/38337.txt',
 '/home/hj20/hmd/murmur/test/40798.txt',
 '/home/hj20/hmd/murmur/test/49558.txt',
 '/home/hj20/hmd/murmur/test/49572.txt',
 '/home/hj20/hmd/murmur/test/49607.txt',
 '/home/hj20/hmd/murmur/test/49628.txt',
 '/home/hj20/hmd/murmur/test/49641.txt',
 '/home/hj20/hmd/murmur/test/49653.txt',
 '/home/hj20/hmd/murmur/test/49683.txt',
 '/home/hj20/hmd/murmur/test/49691.txt',
 '/home/hj20/hmd/murmur/test/49761.txt',
 '/home/hj20/hmd/murmur/test/49824.txt',
 '/home/hj20/hmd/murmur/test/49829.txt',
 '/home/hj20/hmd/murmur/test/49832.txt',
 '/home/hj20/hmd/murmur/test/49896.txt',
 '/home/hj20/hmd/murmur/test/49952.txt',
 '/home/hj20/hmd/murmur/test/49978.txt',
 '/home/hj20/hmd/murmur/test/49980.txt',
 '/home/hj20/hmd/murmur/test/49983.txt',
 '/home/hj20/hmd/murmur/test/50006.txt',
 '/home/hj20/hmd/murmur/test/50012.txt',
 '/home/hj20/hmd/murmur/test/50026.txt',
 '/home/hj20/hmd/murmur/test/50047.txt',
 '/home/hj20/hmd

In [159]:
#사전 학습 모델은 별로였다
def get_features_vgg(data_folder, patient_files_trn) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    murmurs=[] 
    outcomes=[]

    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in tqdm(range(num_patient_files)):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            mel1 = vgg_feature(filename)
            features['mel1'].append(mel1)

            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels and use one-hot encoding.
            current_murmur = np.zeros(num_murmur_classes, dtype=int)
            murmur = get_murmur(current_patient_data)
            if murmur in murmur_classes:
                j = murmur_classes.index(murmur)
                current_murmur[j] = 1
            murmurs.append(current_murmur)
            
            
            current_outcome = np.zeros(num_outcome_classes, dtype=int)
            outcome = get_outcome(current_patient_data)
            if outcome in outcome_classes:
                j = outcome_classes.index(outcome)
                current_outcome[j] = 1
            outcomes.append(current_outcome)
   
    
    #murmurs = np.vstack(murmurs)
    #outcomes = np.vstack(outcomes)
    
    
    #with open('./vgg_128_features.pickle', 'wb') as f:
    #    pickle.dump(features, f)        
            
            
    
    #M, N = features['mel1'][i].shape
    #for i in range(len(features['mel1'])) :
    #    features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    murmurs = np.array(murmurs)
    outcomes = np.array(outcomes)
    
    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
   #저장하기  
    #np.save('./vgg_feature', features)
    #np.save('./vgg_murmurs', murmurs)
    #np.save('./vgg_outcomes', outcomes)
    
    
    return features, murmurs, outcomes

In [15]:
import pickle

In [16]:
with open('./vgg_128_features.pickle', 'wb') as f:
    pickle.dump(features, f)  

NameError: name 'features' is not defined

In [17]:
with open('./vgg_128_murmurs.pickle', 'wb') as f:
    pickle.dump(murmurs, f)  

NameError: name 'murmurs' is not defined

In [18]:
with open('./vgg_128_outcomes.pickle', 'wb') as f:
    pickle.dump(outcomes, f)  

NameError: name 'outcomes' is not defined

In [27]:
features,murmurs, outcomes= get_features_vgg(data_folder, patient_files)

100%|██████████| 751/751 [02:15<00:00,  5.53it/s]


In [28]:
features[2]

array([[-0.01485951, -0.00024288,  0.01596607, ..., -0.00932899,
         0.0062453 ,  0.00283915],
       [-0.01491961, -0.00025762,  0.01595007, ..., -0.00950889,
         0.00607612,  0.00288117],
       [-0.01477804, -0.00056635,  0.01597399, ..., -0.00939149,
         0.00606144,  0.00278954],
       ...,
       [-0.01495733, -0.00096725,  0.01601251, ..., -0.00988272,
         0.00580987,  0.00338692],
       [-0.01467432, -0.0002003 ,  0.0156535 , ..., -0.00968187,
         0.00615748,  0.00266432],
       [-0.01467718, -0.00011551,  0.01585453, ..., -0.00980829,
         0.00655373,  0.00382918]], dtype=float32)

In [29]:
#외부 데이터 사용 안하기 
import numpy as np
import torch
import sys
import os
sys.path.append("..")
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn as nn


from scipy.io import wavfile
import random


class myDataset(Dataset):
    def __init__(self, x_npy, y_npy):    # spec or mel_spec
        self.x_npy = x_npy
        self.y_npy = y_npy

        
    def __getitem__(self, idx):
        cur_idx = idx
        while 1:
            try:
                data_feature = self.x_npy[cur_idx]
                #age = self.x_npy['age'][cur_idx]
                #sex = self.x_npy['sex'][cur_idx]
                #hw = self.x_npy['hw'][cur_idx]
                #if self.x_npy['preg'][cur_idx] :
                #    preg = np.array([1])
                #else:
                #    preg = np.array([0])
                
                #loc = self.x_npy['loc'][cur_idx]
                break
            except:
                cur_idx += 1 
        
        data_label = self.y_npy[cur_idx]
        return data_feature , data_label

    def __len__(self):
        return len(self.x_npy)

def myCollateFn(sample_batch):
    sample_batch = sorted(sample_batch, key=lambda x: x[0].shape[0], reverse=True)
    #data_feature = [spec_augment(torch.from_numpy(x[0])) for x in sample_batch]#spec
    data_feature =[torch.from_numpy(x[0]) for x in sample_batch]
    #age = [torch.from_numpy(x[1]) for x in sample_batch]
    #sex = [torch.from_numpy(x[2]) for x in sample_batch]
    #hw = [torch.from_numpy(x[3]) for x in sample_batch]
    #preg = [torch.from_numpy(x[4]) for x in sample_batch]
    #loc = [torch.from_numpy(x[5]) for x in sample_batch]
    
    data_feature = pad_sequence(data_feature, batch_first=True, padding_value=0.0)
    #age = pad_sequence(age, batch_first=False, padding_value=0.0)
    #sex = pad_sequence(sex, batch_first=False, padding_value=0.0)
    #hw = pad_sequence(hw, batch_first=False, padding_value=0.0)
    #preg = pad_sequence(preg, batch_first=False, padding_value=0.0)
    #loc = pad_sequence(loc, batch_first=False, padding_value=0.0)
    
    data_label = torch.tensor([x[-1] for x in sample_batch])
    return data_feature , data_label

class myDataLoader(DataLoader):
    def __init__(self, *args, **kwargs):
        super(myDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = myCollateFn


In [69]:
torch.tensor(murmurs)

tensor([[0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]])

In [27]:
features = np.load('vgg_128_features.pickle',allow_pickle=True)

OSError: Failed to interpret file 'vgg_128_features.pickle' as a pickle

In [29]:
import pickle

with open('vgg_128_features.pickle', 'rb') as file:
    features = pickle.load(file)

EOFError: Ran out of input

In [5]:
murmurs = np.load('vgg_128_murmurs.pickle',allow_pickle=True)

OSError: Failed to interpret file 'vgg_128_murmurs.pickle' as a pickle

In [30]:
dataset_train = myDataset(features, murmurs)
dataloader_train = myDataLoader(dataset=dataset_train,
                                    batch_size=16,
                                    shuffle=False,
                                    num_workers=4)

In [226]:
len(features)

2532

In [120]:
dataset_train = myDataset(features, murmurs)

In [121]:
dataloader_train = myDataLoader(dataset=dataset_train,
                                    batch_size=64,
                                    shuffle=False,
                                    num_workers=0)

In [31]:
len(dataloader_train)

159

In [141]:
dataset_train[0] 

(array([[ 0.00707377,  0.00185162,  0.00473471, ..., -0.00871591,
         -0.00792329,  0.01029355],
        [ 0.00644913,  0.0017787 ,  0.00425978, ..., -0.00887532,
         -0.00759107,  0.01025419],
        [ 0.00641653, -0.00011679,  0.00565482, ..., -0.0077928 ,
         -0.00809923,  0.01015981],
        ...,
        [ 0.00671153,  0.00078823,  0.00479199, ..., -0.00914858,
         -0.00766064,  0.01080061],
        [ 0.00617167,  0.00055305,  0.00609201, ..., -0.00813335,
         -0.00821544,  0.01012022],
        [ 0.00645245,  0.00107319,  0.0049415 , ..., -0.00916245,
         -0.00780882,  0.00996371]], dtype=float32),
 array([0, 0, 1]))

In [ ]:
def train():
    model.train()
    total_loss_text = 0.
    start_time = time.time()
    batch = 0
    for src, tgt, tgt_len, ref in training_data:
        src = src.to(device)
        tgt = tgt.to(device)
        tgt_pad_mask = get_padding(tgt, tgt_len)
        tgt_in = tgt[:, :-1]
        tgt_pad_mask = tgt_pad_mask[:, :-1]
        tgt_y = tgt[:, 1:]

        optimizer.zero_grad()
        output = model(src, tgt_in, target_padding_mask=tgt_pad_mask)

        loss_text = criterion(output.contiguous().view(-1, hp.ntoken), tgt_y.transpose(0, 1).contiguous().view(-1))
        loss = loss_text
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), hp.clip_grad)
        optimizer.step()
        total_loss_text += loss_text.item()

        writer.add_scalar('Loss/train-text', loss_text.item(), (epoch - 1) * len(training_data) + batch)

        batch += 1

        if batch % hp.log_interval == 0 and batch > 0:
            mean_text_loss = total_loss_text / hp.log_interval
            elapsed = time.time() - start_time
            current_lr = [param_group['lr'] for param_group in optimizer.param_groups][0]
            logging.info('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2e} | ms/batch {:5.2f} | '
                         'loss-text {:5.4f}'.format(
                epoch, batch, len(training_data), current_lr,
                elapsed * 1000 / hp.log_interval, mean_text_loss))
            total_loss_text = 0
            start_time = time.time()

In [13]:
import torch
import torch.nn as nn
import time

from data_handling import get_clotho_loader, get_test_data_loader
from model import TransformerModel  # , RNNModel, RNNModelSmall
import itertools
import numpy as np
import os
import sys
import logging
import csv

ModuleNotFoundError: No module named 'data_handling'

In [24]:
from torch.utils.tensorboard import SummaryWriter

In [71]:
features = np.vstack(features)

In [74]:
features.shape

(59155, 128)

In [76]:
features.shape

(59155, 128)

In [80]:
mel1 = vgg_feature(filename)

In [82]:
mel1.shape

(18, 128)

In [36]:
audio_feature, age, sex, hw, preg, loc, data_label_torch =next(iter(dataloader_train)) 

In [37]:
audio_feature.shape #(b,t,bin)

torch.Size([64, 34, 128])

In [38]:
data_label_torch[0]

tensor([[1],
        [0],
        [0]])

## 모델 방법론들

### cnn 모델

In [32]:
#cnn 모델 
#from tensorflow import keras
#from tensorflow.keras import layers

# -*- coding: utf-8 -*-
import argparse
import os
import time
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import sys
import random
from tqdm import tqdm
from torch import Tensor
import torch.nn.functional as F



def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)
 
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
    
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(1, 1), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(1, 1), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        
        return x

    
class vgg_hj(nn.Module):
    def __init__(self, classes_num):
        
        super(vgg_hj, self).__init__() 
        


        self.bn0 = nn.BatchNorm2d(128)
        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        #self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        #self.dense_age = Dense(6)
        #self.dense_sex = Dense(2)
        #self.dense_hw = Dense(2)
        #self.dense_preg = Dense(1)
        #self.dense_loc = Dense(5)
        
        #self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024) 
        #self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        #self.fc1 = nn.Linear(512, 507, bias=True)
        self.fc_audioset = nn.Linear(256, classes_num, bias=True)
        #self.lstm = nn.LSTM(
        #    input_size=512,
        #    hidden_size=512,
        #    num_layers=1,bias=True,bidirectional=False,batch_first= True
        #)
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        #init_layer(self.fc1)
        init_layer(self.fc_audioset)
        
        
    def forward(self, input):

        """
        Input: (batch_size, data_length)"""

        #x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        #x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        #print(input.shape)
        #x = input.permute(1, 2, 0) 
        input = input.float()
        x =input.unsqueeze(1)
        #x = input.permute(0, 3,1, 2) 
        #print(x.shape)
        x = x.transpose(1, 3)
        #print(x.shape)
        x = self.bn0(x)
        #print(x.shape)
        #x = x.permute(0, 2,1, 3) 
        x = x.transpose(1, 3)
        #print(x.shape)
        
        x = self.conv_block1(x, pool_size=(1, 1), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.1, training=self.training)
        #print(x.shape)
        x = self.conv_block2(x, pool_size=(3, 3), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.1, training=self.training)
        #print(x.shape)
        x = self.conv_block3(x, pool_size=(3, 3), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.1, training=self.training)
        #print(x.shape)
        #x = self.conv_block4(x, pool_size=(1, 1), pool_type='avg')
        #print(x.shape)
        #x = F.dropout(x, p=0.2, training=self.training)
        #x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        #x = F.dropout(x, p=0.2, training=self.training)
        #x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        #x = F.dropout(x, p=0.2, training=self.training)
        #print(x.shape)
        #print(x.shape)
        x = torch.mean(x, dim=3)
        #print(x)
        #print(x.shape)
        (x1, _) = torch.max(x, dim=2)
        #print(x1.shape)
        x2 = torch.mean(x, dim=2)
        #print(x2.shape)
        x = x1 + x2
        #print(x.shape)
        x = F.dropout(x, p=0.1, training=self.training)
        #print(x.shape)
        #x = F.relu_(self.fc1(x))
        #print('fc통과:',x.shape)
        #_,(hidden, _) = self.lstm(x.transpose(1,2))
        
        #lstm_out = hidden[-1]
        #print(lstm_out.shape)
        
        out = self.fc_audioset(x)
    
        #out = F.relu_(self.fc_audioset(x))
        #out = self.fc_audioset(lstm_out)
        #print('out통과:',out.shape)
        #clipwise_output = torch.sigmoid(out)

        return out


In [66]:
audio_feature.shape

torch.Size([64, 34, 128])

In [67]:
x =audio_feature.unsqueeze(1)

In [68]:
x.shape

torch.Size([64, 1, 34, 128])

In [69]:
x = x.transpose(1, 3)

In [70]:
x.shape

torch.Size([64, 128, 34, 1])

In [71]:
bn0 = nn.BatchNorm2d(128)

In [72]:
x = bn0(x)

In [73]:
x.shape

torch.Size([64, 128, 34, 1])

In [74]:
x = x.transpose(1, 3)

In [75]:
x.shape

torch.Size([64, 1, 34, 128])

### attention 기반 cnn

In [ ]:
class Cnn14_DecisionLevelAtt(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, 
        fmax, classes_num):
        
        super(Cnn14_DecisionLevelAtt, self).__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        self.interpolate_ratio = 32     # Downsampled ratio

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.att_block = AttBlock(2048, classes_num, activation='sigmoid')
        
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
 
    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)

        frames_num = x.shape[2]
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.mean(x, dim=3)
        
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        (clipwise_output, _, segmentwise_output) = self.att_block(x)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output, self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        output_dict = {'framewise_output': framewise_output, 
            'clipwise_output': clipwise_output}

        return output_dict

In [ ]:
class AttBlock(nn.Module):
    def __init__(self, n_in, n_out, activation='linear', temperature=1.):
        super(AttBlock, self).__init__()
        
        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(in_channels=n_in, out_channels=n_out, kernel_size=1, stride=1, padding=0, bias=True)
        self.cla = nn.Conv1d(in_channels=n_in, out_channels=n_out, kernel_size=1, stride=1, padding=0, bias=True)
        
        self.bn_att = nn.BatchNorm1d(n_out)
        self.init_weights()
        
    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)
         
    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.clamp(self.att(x), -10, 10), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)


### cnn-lstm

In [233]:
class CNN1dLSTM(nn.Module):
    def __init__(self, n_features, n_hidden, n_layers):
        super(CNN1dLSTM, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.c1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size = 3, stride = 1) # 1D CNN 레이어 추가
        self.lstm = nn.LSTM(
            input_size=512,
            hidden_size=256,
            num_layers=2,bias=True,bidirectional=False,batch_first= True
        )
        self.fc_layer1 = nn.Linear(in_features=n_hidden, out_features=3)

    def forward(self,x,length=None):
        x = x.unsqueeze(1)
        #print(x.shape)
        batch_size,C,time_step=x.size()
        
        cnn_out = self.c1(x)
        cnn_out = cnn_out.transpose(1,2)
        
        _, (hidden, _) = self.lstm(cnn_out)
        lstm_out = hidden[-1]
        
        out = self.fc_layer1(lstm_out)
        return out

In [234]:
model = CNN1dLSTM(
    n_features=4,
    n_hidden=8,
    n_layers=1
)

In [235]:
model = model.cuda()

In [218]:
model(audio_feature).shape

torch.Size([8, 1, 106752])


torch.Size([8, 3])

In [163]:
c1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size = 3, stride = 1)

In [164]:
c2 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size = 1, stride = 1)

In [189]:
lstm = nn.LSTM(input_size=4,
                            hidden_size=8,
                            num_layers=1,
                            bias=True,
                            bidirectional=False,
                            batch_first=True)


In [165]:
c1 = c1.cuda()

In [166]:
c2 = c2.cuda()

In [174]:
x=x.transpose(1,2)

In [192]:
_, (hidden, _) = lstm(c1(audio_feature).transpose(1,2))

In [196]:
hidden[-1].shape

torch.Size([8, 8])

In [126]:
hidden.shape

torch.Size([1, 16, 8])

In [132]:
x = hidden[-1]

In [133]:
x.shape

torch.Size([16, 8])

In [178]:
c1(audio_feature).shape

torch.Size([16, 4, 115134])

In [182]:
audio_feature

tensor([[ 12672,  10041,   3215,  ...,   -658,      8,    528],
        [  -425,   1045,    518,  ...,      0,      0,      0],
        [   244,  -3588, -22579,  ...,      0,      0,      0],
        ...,
        [  1653,   3815,   3101,  ...,      0,      0,      0],
        [  2276,   2343,   2448,  ...,      0,      0,      0],
        [  2593,   1874,   1370,  ...,      0,      0,      0]],
       dtype=torch.int16)

In [169]:
x = c2(c1(audio_feature))

In [170]:
x.shape

torch.Size([16, 8, 115134])

In [155]:
x= x.transpose(1,2)

In [ ]:
x = torch.mean(x, dim=3)
#print(x)
#print(x.shape)
(x1, _) = torch.max(x, dim=2)
#print(x1.shape)
x2 = torch.mean(x, dim=2)
#print(x2.shape)
x = x1 + x2

In [156]:
(x1, _) = torch.max(x, dim=2)

In [157]:
x1.shape

torch.Size([16, 115134])

In [158]:
x2 = torch.mean(x, dim=2)

In [159]:
x2.shape

torch.Size([16, 115134])

In [160]:
x = x1 + x2

In [161]:
x.shape

torch.Size([16, 115134])

In [99]:
c1(audio_feature).shape

torch.Size([64, 8, 131392])

In [191]:
lstm = lstm.cuda()

In [113]:
audio_feature.shape

torch.Size([16, 115136])

In [69]:
len(audio_feature)

64

In [197]:
batch_size,C,time_step = audio_feature.size()

In [198]:
batch_size

8

In [73]:
C

1

In [74]:
time_step

131392

In [183]:
audio_feature = audio_feature.unsqueeze(1)

In [184]:
audio_feature =audio_feature.cuda()

In [185]:
audio_feature.shape

torch.Size([8, 1, 106752])

In [41]:
model = model.cuda()

In [187]:
audio_feature =audio_feature.float()

In [70]:
model(audio_feature.cuda())

RuntimeError: Given groups=1, weight of size [1, 64, 2], expected input[64, 1, 131392] to have 64 channels, but got 1 channels instead

## raw feature

In [195]:
for j in range(num_locations) :
    entries = recording_information[j].split(' ')
    recording_file = entries[2]
    filename = os.path.join(data_folder, recording_file)

In [200]:
recording, _ = load_wav_file(filename)

In [201]:
recording

array([ 3184,    73,  2484, ...,  -698, -1320,  1233], dtype=int16)

In [202]:
frequency

4000

In [10]:
features,murmurs, outcomes= get_features_raw(data_folder, patient_files)

100%|██████████| 751/751 [00:00<00:00, 2226.50it/s]
/home/hj20/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [48]:
features

{'id': array(['2530', '2530', '2530', ..., '85349', '85349', '85349'],
       dtype='<U5'),
 'age': array([[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1]]),
 'sex': array([[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]]),
 'hw': array([[98. , 15.9],
        [98. , 15.9],
        [98. , 15.9],
        ...,
        [ nan,  nan],
        [ nan,  nan],
        [ nan,  nan]]),
 'preg': array([False, False, False, ...,  True,  True,  True]),
 'loc': array([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        ...,
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]]),
 'mel1': array([array([-425, 1045,  518, ...,  509,  443,  122], dtype=int16),
        array([2593, 1874, 1370, ..., -637, -255, -243], dtype=int16),
        array([ 2276,  2343,  

In [27]:
audio_feature, age, sex, hw, preg, loc, data_label_torch = next(iter(dataloader_train_murmurs))

In [29]:
audio_feature.shape

torch.Size([64, 131392])

In [227]:
model=model.cuda()

In [237]:
#imputer = SimpleImputer().fit(features['mel1'])
#features['mel1'] = imputer.transform(features['mel1'])
log_dir = './log/'
project = 'hj0'
logger = get_logger(log_dir + '/' + project)
    

    
for iter_ in range(50):  # args.end_iter
    start_time = time.time()
    running_loss = 0.0
    train_epoch_acc = 0.0
    train_acc = 0.0
    model.train()
    labels=[]
    idx = 0
#    epoch[str(iter_)] = {}
    for audio_feature, age, sex, hw, preg, loc, data_label_torch in tqdm(dataloader_train_murmurs):  
            #print(audio_feature.shape)
        
        #idx = idx + 1
        #audio_feature = torch.nan_to_num(audio_feature, nan=0)
        audio_feature[audio_feature != audio_feature] = 0
        
        #audio_feature = audio_feature.permute(1,3,0,2)
        #print(audio_feature.shape)
        audio_feature = audio_feature.float()
        audio_feature = audio_feature.cuda()
        #age = age.permute(1,0).float().cuda()
            #print(age.shape)
        #    sex = sex.permute(1,0).float().cuda()
            #print(sex.shape)
        #    hw = hw.permute(1,0).float().cuda()
            #print(hw.shape)
        #    preg = preg.permute(1,0).float().cuda()
            #print(preg.shape)
         #   loc = loc.permute(1,0).float().cuda()
            #print(loc.shape)
    #        for idx in range(len(data_label_torch)) :
    #            data_label_torch[idx][0] = mapping[int(data_label_torch[idx][0])]
        data_label_torch = data_label_torch.cuda()
                
        data_label_torch = data_label_torch.long()
        outputs = model(x=audio_feature)
            
        #softmax = nn.Softmax(dim = 1)
            
        #outputs = softmax(outputs)
            
        #labels = np.argmax(outputs.detach().cpu().numpy())
        murmur_labels = np.zeros((audio_feature.shape[0],len(murmur_classes)), dtype=np.int_)
        for i in range(audio_feature.shape[0]):
            idx = np.argmax(outputs[i].detach().cpu().numpy())
            murmur_labels[i][idx] = 1 
            
            
        labels = torch.argmax(data_label_torch, 1)    
            
        loss = criterion(outputs.float(),labels)
        #train_acc = compute_weighted_accuracy(data_label_torch.squeeze(1), outputs,murmur_classes )
        #print(outputs.shape)
        #print(murmur_labels.shape)
            
        train_acc = compute_weighted_accuracy(murmur_labels, data_label_torch.detach().cpu().numpy(),murmur_classes)
        #print(train_acc)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 2.5)
        optimizer.step()

        running_loss += loss.item()
        train_epoch_acc += train_acc.item()
            #if idx % 100 == 0 :
            #    print("lr: ", optimizer.param_groups[0]['lr'])
            #    print('batch : ',idx, ' done')
    logging.info("Iteration:{0}, loss = {1:.6f}, train_acc = {2:.3f} ".format(iter_, running_loss/all_file_train, train_epoch_acc/all_file_train))
    
save_challenge_model(model_folder, murmur_classes, model)
                

NameError: name 'dataloader_train_murmurs' is not defined

In [241]:
all_file_train

633

In [237]:
!nvidia-smi

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

Wed Jul 20 11:28:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 40%   49C   

In [199]:
def load_recordings(data_folder, data, get_frequencies=False):
    num_locations = get_num_locations(data)
    recording_information = data.split('\n')[1:num_locations+1]

    recordings = list()
    frequencies = list()
    for i in range(num_locations):
        entries = recording_information[i].split(' ')
        recording_file = entries[2]
        filename = os.path.join(data_folder, recording_file)
        recording, frequency = load_wav_file(filename)
        recordings.append(recording)
        frequencies.append(frequency)

    if get_frequencies:
        return recordings, frequencies
    else:
        return recordings

TypeError: load_recordings() missing 1 required positional argument: 'data'

In [4]:
def get_features_raw(data_folder, patient_files_trn) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    murmurs=[] 
    outcomes=[]

    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in tqdm(range(num_patient_files)):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            recording, _ = load_wav_file(filename)
            features['mel1'].append(recording)

            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels and use one-hot encoding.
            current_murmur = np.zeros(num_murmur_classes, dtype=int)
            murmur = get_murmur(current_patient_data)
            if murmur in murmur_classes:
                j = murmur_classes.index(murmur)
                current_murmur[j] = 1
            murmurs.append(current_murmur)
            
            
            current_outcome = np.zeros(num_outcome_classes, dtype=int)
            outcome = get_outcome(current_patient_data)
            if outcome in outcome_classes:
                j = outcome_classes.index(outcome)
                current_outcome[j] = 1
            outcomes.append(current_outcome)
   
    
    #murmurs = np.vstack(murmurs)
    #outcomes = np.vstack(outcomes)
    
    
    #with open('./vgg_128_features.pickle', 'wb') as f:
    #    pickle.dump(features, f)        
            
            
    
    #M, N = features['mel1'][i].shape
    #for i in range(len(features['mel1'])) :
    #    features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    murmurs = np.array(murmurs)
    outcomes = np.array(outcomes)
    
    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
   #저장하기  
    #np.save('./vgg_feature', features)
    #np.save('./vgg_murmurs', murmurs)
    #np.save('./vgg_outcomes', outcomes)
    
    
    return features, murmurs, outcomes

# train model

In [185]:
#사전학습네트워크 피쳐 로드 하기 
features =np.load('./vgg_feature.npy',allow_pickle=True)
murmurs=np.load('./vgg_murmurs.npy',allow_pickle=True)
outcomes=np.load('./vgg_outcomes.npy',allow_pickle=True)

In [38]:
murmur_classes = ['Present', 'Unknown', 'Absent']
num_murmur_classes = len(murmur_classes)
outcome_classes = ['Abnormal', 'Normal']
num_outcome_classes = len(outcome_classes)
    
    
#features = np.vstack(features)
#murmurs = np.vstack(murmurs)
#outcomes = np.vstack(outcomes)
# Train the model.
log_dir = './log/'
project = 'hj0'
logger = get_logger(log_dir + '/' + project)
    




In [18]:
features['mel1']

array([array([-425, 1045,  518, ...,  509,  443,  122], dtype=int16),
       array([2593, 1874, 1370, ..., -637, -255, -243], dtype=int16),
       array([ 2276,  2343,  2448, ..., -2732, -5634, -4290], dtype=int16),
       ...,
       array([    2,   128,   481, ..., -1761, -1536,  -348], dtype=int16),
       array([4959, 2317, 4913, ..., -831, -365, -199], dtype=int16),
       array([7055, 3533,  107, ..., 1380, 1053, 1048], dtype=int16)],
      dtype=object)

In [27]:
murmurs_label

array([2., 2., 2., ..., 2., 2., 2.])

In [18]:
#label 
murmurs_label = np.empty(len(murmurs))
for i,l in enumerate(murmurs) :
    murmurs_label[i] = int(list(l).index(1))
    
outcomes_label = np.empty(len(outcomes))
for i,l in enumerate(outcomes) :
    outcomes_label[i] = int(list(l).index(1))
    
    

In [97]:
murmurs_label.shape

(2532,)

In [21]:
dataset_train_murmurs = myDataset(features,murmurs)

In [238]:
dataloader_train_murmurs = myDataLoader(dataset=dataset_train_murmurs,
                                    batch_size=4,
                                    shuffle=False,
                                    num_workers=0)

In [238]:
features['mel1'][2].shape

(59968,)

In [23]:
len(dataloader_train_murmurs)

40

In [ ]:
dataloader_train_murmurs

In [243]:
features['mel1'][0]

array([-425, 1045,  518, ...,  509,  443,  122], dtype=int16)

In [261]:
conv1d = nn.Conv1d(1, 512, kernel_size=3)

In [252]:
x = features['mel1'][0].reshape(1,-1)

In [270]:
x = x.float()

In [266]:
conv1d = conv1d.cuda()

In [272]:
conv1d(x).shape

torch.Size([1, 512, 94398])

In [212]:
len(dataset_train_murmurs)

NameError: name 'dataset_train_murmurs' is not defined

In [173]:
dataset_train_murmurs = myDataset(features,murmurs)
dataloader_train_murmurs = myDataLoader(dataset=dataset_train_murmurs,
                                    batch_size=64,
                                    shuffle=False,
                                    num_workers=0)
    
    
dataset_train_outcomes = myDataset(features,outcomes_label)
dataloader_train_outcoms = myDataLoader(dataset=dataset_train_outcomes,
                                    batch_size=64,
                                    shuffle=False,
                                    num_workers=0)

all_file_train = len(dataloader_train_murmurs)

In [39]:
all_file_train = len(dataloader_train)

In [187]:
nnet = vgg_hj(num_murmur_classes) 

In [188]:
nnet = nnet.cuda()

In [189]:
# Define model
#nnet = vgg_hj(num_murmur_classes) 
#nnet = nnet.cuda()
optimizer = optim.Adam(nnet.parameters(), lr=4e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.91)
criterion = nn.CrossEntropyLoss()


In [49]:
# Train your model.
def train_challenge_model(data_folder, model_folder, verbose):
    gc.collect()
    torch.cuda.empty_cache()
    GPU_NUM = 0 # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
#    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device)
    # Find data files.
    if verbose >= 1:
        print('Finding data files...')

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the model if it does not already exist.
    os.makedirs(model_folder, exist_ok=True)

    # Extract the features and labels.
    if verbose >= 1:
        print('Extracting features and labels from the Challenge data...')

    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    
    #features = np.vstack(features)
    #murmurs = np.vstack(murmurs)
    #outcomes = np.vstack(outcomes)
    # Train the model.
    log_dir = './log/'
    project = 'hj0'
    logger = get_logger(log_dir + '/' + project)
    
    #label 
    murmurs_label = np.empty(len(murmurs))
    for i,l in enumerate(murmurs) :
        murmurs_label[i] = int(list(l).index(1))
    
    outcomes_label = np.empty(len(outcomes))
    for i,l in enumerate(outcomes) :
        outcomes_label[i] = int(list(l).index(1))
    
    
    dataset_train_murmurs = myDataset(features,murmurs_label)
    dataloader_train_murmurs = myDataLoader(dataset=dataset_train_murmurs,
                                    batch_size=64,
                                    shuffle=False,
                                    num_workers=0)
    
    
    dataset_train_outcomes = myDataset(features,outcomes_label)
    dataloader_train_outcoms = myDataLoader(dataset=dataset_train_outcomes,
                                    batch_size=64,
                                    shuffle=False,
                                    num_workers=0)

    all_file_train = len(dataloader_train_murmurs)
    # Define model
    nnet = vgg_hj(num_murmur_classes) 
    nnet = nnet.cuda()
    optimizer = optim.Adam(nnet.parameters(), lr=4e-4)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.91)
    criterion = nn.CrossEntropyLoss()
    # Train the model.
    if verbose >= 1:
        print('Training model...')
    imputer = SimpleImputer().fit(features['hw'])
    features['hw'] = imputer.transform(features['hw'])
    
    
    for iter_ in range(15):  # args.end_iter
        start_time = time.time()
        running_loss = 0.0
        train_epoch_acc = 0.0
        nnet.train()
        labels=[]
        idx = 0
#    epoch[str(iter_)] = {}
        for audio_feature, age, sex, hw, preg, loc, data_label_torch in tqdm(dataloader_train_murmurs):  
            #print(audio_feature.shape)
        
            idx = idx + 1
            #audio_feature = torch.nan_to_num(audio_feature, nan=0)
            audio_feature[audio_feature != audio_feature] = 0
        
            #audio_feature = audio_feature.permute(1,3,0,2)
            #print(audio_feature.shape)
            audio_feature = audio_feature.cuda()
            age = age.permute(1,0).float().cuda()
            #print(age.shape)
            sex = sex.permute(1,0).float().cuda()
            #print(sex.shape)
            hw = hw.permute(1,0).float().cuda()
            #print(hw.shape)
            preg = preg.permute(1,0).float().cuda()
            #print(preg.shape)
            loc = loc.permute(1,0).float().cuda()
            #print(loc.shape)
    #        for idx in range(len(data_label_torch)) :
    #            data_label_torch[idx][0] = mapping[int(data_label_torch[idx][0])]
            data_label_torch = data_label_torch.cuda()
                
            data_label_torch = data_label_torch.long()
            outputs = nnet(input=audio_feature, age = age , sex = sex, hw = hw, preg = preg, loc = loc)
            
            #softmax = nn.Softmax(dim = 1)
            
            #outputs = softmax(outputs)
            
            #labels = np.argmax(outputs.detach().cpu().numpy())
            
            
            loss = criterion(outputs.float(), data_label_torch.squeeze(1))
            #train_acc = compute_weighted_accuracy(data_label_torch.squeeze(1), outputs,murmur_classes )
            train_acc = (outputs, data_label_torch.squeeze(1))
            #print(outputs.shape)
            #print(data_label_torch.squeeze(1).shape)
            #print(train_acc)
                
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_epoch_acc += train_acc.item()
            #if idx % 100 == 0 :
            #    print("lr: ", optimizer.param_groups[0]['lr'])
            #    print('batch : ',idx, ' done')
        logger.info("Iteration:{0}, loss = {1:.6f}, acc = {2:.3f} ".format(iter_, running_loss/all_file_train, train_epoch_acc/all_file_train))
    
    save_challenge_model(model_folder, murmur_classes, nnet,m_name='vgg_hj')
                

In [37]:
outputs = nnet(input=audio_feature)

torch.Size([64, 1, 34, 128])
tensor([[[[0.7365, 0.1450, 0.3053,  ..., 0.3276, 0.8542, 0.0133],
          [0.1623, 0.1162, 0.2642,  ..., 0.5382, 0.1436, 0.0000],
          [0.0000, 0.0000, 0.2549,  ..., 0.2423, 1.1088, 0.4435],
          ...,
          [1.4354, 1.3017, 0.3659,  ..., 1.2430, 0.0000, 0.5467],
          [0.0000, 0.5971, 0.5348,  ..., 0.5506, 0.0000, 0.6803],
          [0.3304, 0.4733, 0.4829,  ..., 0.0000, 0.1462, 0.0000]],

         [[0.0000, 0.0000, 0.5742,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.6671, 0.2682,  ..., 0.7510, 0.7123, 0.0709],
          [0.2528, 0.4811, 0.3006,  ..., 0.2794, 0.7664, 0.4984],
          ...,
          [0.5715, 0.5405, 0.8981,  ..., 0.0000, 0.0000, 0.1495],
          [0.3716, 1.8294, 1.1012,  ..., 1.2264, 0.3704, 0.2962],
          [0.0000, 0.0000, 0.3975,  ..., 0.6722, 0.4177, 0.4407]],

         [[0.1315, 1.0866, 0.0000,  ..., 0.6944, 0.0000, 0.0000],
          [0.0000, 0.5627, 0.0000,  ..., 0.1419, 0.2632, 0.4772],
          [0.

In [38]:
outputs[0]

tensor([-1.4026,  1.7008,  3.2087], device='cuda:0', grad_fn=<SelectBackward>)

In [135]:
audio_feature.shape

torch.Size([64, 34, 128])

In [77]:
audio_feature.shape[0]

36

In [117]:
labels = torch.argmax(data_label_torch, 1)  

In [121]:
outputs = torch.argmax(data_label_torch, 1) 

In [123]:
 outputs = nnet(input=audio_feature)

In [127]:
torch.argmax(outputs, 1) 

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2,
        2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')

In [138]:
torch.argmax(softmax(outputs),1)

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2,
        2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')

In [136]:
softmax = nn.Softmax(dim = 1)

In [54]:
outputs.shape

torch.Size([64, 3])

In [56]:
data_label_torch.squeeze(-1).shape

torch.Size([64, 3])

In [ ]:
outputs = nnet(input=audio_feature)

In [63]:
audio_feature, age, sex, hw, preg, loc, data_label_torch in tqdm(dataloader_train)

100%|██████████| 40/40 [00:00<00:00, 588.31it/s]


(tensor([[[-0.0870,  0.2871,  0.3189,  ..., -0.1034,  0.0943, -0.1561],
          [-0.2312,  0.4577,  0.2643,  ..., -0.3078,  0.1330, -0.0348],
          [-0.3348,  0.1452,  0.1371,  ..., -0.1925,  0.2466, -0.1260],
          ...,
          [-0.2951,  0.3147,  0.3223,  ..., -0.2615,  0.0862, -0.0267],
          [-0.4096,  0.2356,  0.1309,  ..., -0.5459,  0.0485, -0.2727],
          [-0.2912,  0.1199,  0.2163,  ..., -0.7438, -0.0904, -0.6289]],
 
         [[-0.1507,  0.2252,  0.1186,  ..., -0.3443, -0.0027, -0.2517],
          [-0.1101, -0.2240, -0.0030,  ..., -0.2908, -0.2757, -0.2910],
          [-0.1055,  0.1654,  0.3138,  ..., -0.3962, -0.0092, -0.3875],
          ...,
          [-0.0500,  0.0577,  0.2271,  ..., -0.1455,  0.0986, -0.0856],
          [-0.1688,  0.2108,  0.1663,  ..., -0.0651,  0.2703, -0.1869],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.6866,  0.0826,  0.3639,  ..., -0.8057, -0.3007, -0.2747],
          [-0.5121,  0.1986,

In [32]:
nnet(input=audio_feature)

NameError: name 'audio_feature' is not defined

In [83]:
nnet

vgg_hj(
  (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [108]:
#imputer = SimpleImputer().fit(features['mel1'])
#features['mel1'] = imputer.transform(features['mel1'])
log_dir = './log/'
project = 'hj0'
logger = get_logger(log_dir + '/' + project)
torch.cuda.empty_cache()
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
#    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
    # Find data files.    

    
for iter_ in range(30):  # args.end_iter
    CUDA_LAUNCH_BLOCKING="1"
    start_time = time.time()
    running_loss = 0.0
    train_epoch_acc = 0.0
    train_acc = 0.0
    nnet.train()
    labels=[]
    prob_=[]
    idx = 0
#    epoch[str(iter_)] = {}
    for audio_feature, age, sex, hw, preg, loc, data_label_torch in tqdm(dataloader_train_murmurs):  
            #print(audio_feature.shape)
        
        #idx = idx + 1
        #audio_feature = torch.nan_to_num(audio_feature, nan=0)
        audio_feature[audio_feature != audio_feature] = 0
        
        #audio_feature = audio_feature.permute(1,3,0,2)
        #print(audio_feature.shape)
        audio_feature = audio_feature.cuda()
        #age = age.permute(1,0).float().cuda()
            #print(age.shape)
        #    sex = sex.permute(1,0).float().cuda()
            #print(sex.shape)
        #    hw = hw.permute(1,0).float().cuda()
            #print(hw.shape)
        #    preg = preg.permute(1,0).float().cuda()
            #print(preg.shape)
         #   loc = loc.permute(1,0).float().cuda()
            #print(loc.shape)
    #        for idx in range(len(data_label_torch)) :
    #            data_label_torch[idx][0] = mapping[int(data_label_torch[idx][0])]
        data_label_torch = data_label_torch.cuda()
                
        data_label_torch = data_label_torch.long()
        outputs = nnet(input=audio_feature)
            
        softmax = nn.Softmax(dim = 1)
            
        prob_ = softmax(outputs)
        
        
        
        
        
        #labels = np.argmax(outputs.detach().cpu().numpy())
        #murmur_labels = np.zeros((audio_feature.shape[0],len(murmur_classes)), dtype=np.int_)
        #for i in range(audio_feature.shape[0]):
        #    idx = np.argmax(outputs[i].detach().cpu().numpy())
        #    murmur_labels[i][idx] = 1 
            
            
        #labels = torch.argmax(prob_, 1)    
            
        loss = criterion(outputs.float(), data_label_torch.squeeze(-1))
        #train_acc = compute_weighted_accuracy(data_label_torch.squeeze(1), outputs,murmur_classes )
        #print(outputs.shape)
        #print(murmur_labels.shape)
        #torch.argmax(softmax(outputs),1    
        train_acc = compute_weighted_accuracy(data_label_torch.detach().cpu().numpy(),torch.argmax(prob_,1).unsqueeze(1).detach().cpu().numpy(),murmur_classes)
        print(train_acc)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        #train_epoch_acc += train_acc.item()
            #if idx % 100 == 0 :
            #    print("lr: ", optimizer.param_groups[0]['lr'])
            #    print('batch : ',idx, ' done')
    logging.info("Iteration:{0}, loss = {1:.6f}, train_acc = {2:.3f} ".format(iter_, running_loss/all_file_train, train_epoch_acc/all_file_train ))
    
save_challenge_model(model_folder, murmur_classes, nnet)
                

  0%|          | 0/40 [00:00<?, ?it/s]


RuntimeError: multi-target not supported at /pytorch/aten/src/THCUNN/generic/ClassNLLCriterion.cu:15

In [78]:
np.unique(data_label_torch.detach().cpu().numpy())

array([0, 1, 2])

In [80]:
data_label_torch.shape

torch.Size([64, 1])

In [106]:
data_label_torch

tensor([0, 2, 2, 1, 0, 2, 2, 1, 2, 1, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 1, 0, 0,
        0, 2, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 2, 0], device='cuda:0')

In [112]:
labels = torch.argmax(data_label_torch, 1)

In [113]:
labels

tensor([0, 2, 2, 1, 0, 2, 2, 1, 2, 1, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 1, 0, 0,
        0, 2, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 2, 0])

## 학습

In [178]:
#imputer = SimpleImputer().fit(features['mel1'])
#features['mel1'] = imputer.transform(features['mel1'])
log_dir = './log/'
project = 'hj0'
logger = get_logger(log_dir + '/' + project)
    

    
for iter_ in range(50):  # args.end_iter
    start_time = time.time()
    running_loss = 0.0
    train_epoch_acc = 0.0
    train_acc = 0.0
    model.train()
    labels=[]
    idx = 0
#    epoch[str(iter_)] = {}
    for audio_feature, age, sex, hw, preg, loc, data_label_torch in tqdm(dataloader_train_murmurs):  
            #print(audio_feature.shape)
        
        #idx = idx + 1
        #audio_feature = torch.nan_to_num(audio_feature, nan=0)
        audio_feature[audio_feature != audio_feature] = 0
        
        #audio_feature = audio_feature.permute(1,3,0,2)
        #print(audio_feature.shape)
        audio_feature = audio_feature.cuda()
        #age = age.permute(1,0).float().cuda()
            #print(age.shape)
        #    sex = sex.permute(1,0).float().cuda()
            #print(sex.shape)
        #    hw = hw.permute(1,0).float().cuda()
            #print(hw.shape)
        #    preg = preg.permute(1,0).float().cuda()
            #print(preg.shape)
         #   loc = loc.permute(1,0).float().cuda()
            #print(loc.shape)
    #        for idx in range(len(data_label_torch)) :
    #            data_label_torch[idx][0] = mapping[int(data_label_torch[idx][0])]
        data_label_torch = data_label_torch.cuda()
                
        data_label_torch = data_label_torch.long()
        outputs = model(input=audio_feature)
            
        #softmax = nn.Softmax(dim = 1)
            
        #outputs = softmax(outputs)
            
        #labels = np.argmax(outputs.detach().cpu().numpy())
        murmur_labels = np.zeros((audio_feature.shape[0],len(murmur_classes)), dtype=np.int_)
        for i in range(audio_feature.shape[0]):
            idx = np.argmax(outputs[i].detach().cpu().numpy())
            murmur_labels[i][idx] = 1 
            
            
        labels = torch.argmax(data_label_torch, 1)    
            
        loss = criterion(outputs.float(),labels)
        #train_acc = compute_weighted_accuracy(data_label_torch.squeeze(1), outputs,murmur_classes )
        #print(outputs.shape)
        #print(murmur_labels.shape)
            
        train_acc = compute_weighted_accuracy(murmur_labels, data_label_torch.detach().cpu().numpy(),murmur_classes)
        #print(train_acc)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 2.5)
        optimizer.step()

        running_loss += loss.item()
        train_epoch_acc += train_acc.item()
            #if idx % 100 == 0 :
            #    print("lr: ", optimizer.param_groups[0]['lr'])
            #    print('batch : ',idx, ' done')
    logging.info("Iteration:{0}, loss = {1:.6f}, train_acc = {2:.3f} ".format(iter_, running_loss/all_file_train, train_epoch_acc/all_file_train))
    
save_challenge_model(model_folder, murmur_classes, model)
                

100%|██████████| 10/10 [00:00<00:00, 18.57it/s]

07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:0, loss = 1.248671, train_acc = 0.465 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:0, loss = 1.248671, train_acc = 0.465 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:0, loss = 1.248671, train_acc = 0.465 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:0, loss = 1.248671, train_acc = 0.465 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:0, loss = 1.248671, train_acc = 0.465 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:0, loss = 1.248671, train_acc = 0.465 



100%|██████████| 10/10 [00:00<00:00, 19.90it/s]

07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:1, loss = 1.025402, train_acc = 0.450 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:1, loss = 1.025402, train_acc = 0.450 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:1, loss = 1.025402, train_acc = 0.450 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:1, loss = 1.025402, train_acc = 0.450 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:1, loss = 1.025402, train_acc = 0.450 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:1, loss = 1.025402, train_acc = 0.450 



100%|██████████| 10/10 [00:00<00:00, 20.84it/s]

...

07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:47, loss = 0.651769, train_acc = 0.671 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:47, loss = 0.651769, train_acc = 0.671 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:47, loss = 0.651769, train_acc = 0.671 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:47, loss = 0.651769, train_acc = 0.671 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:47, loss = 0.651769, train_acc = 0.671 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:47, loss = 0.651769, train_acc = 0.671 



100%|██████████| 10/10 [00:00<00:00, 20.39it/s]

07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:48, loss = 0.665178, train_acc = 0.684 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:48, loss = 0.665178, train_acc = 0.684 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:48, loss = 0.665178, train_acc = 0.684 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:48, loss = 0.665178, train_acc = 0.684 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:48, loss = 0.665178, train_acc = 0.684 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:48, loss = 0.665178, train_acc = 0.684 



100%|██████████| 10/10 [00:00<00:00, 20.44it/s]

07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:49, loss = 0.646950, train_acc = 0.668 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:49, loss = 0.646950, train_acc = 0.668 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:49, loss = 0.646950, train_acc = 0.668 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:49, loss = 0.646950, train_acc = 0.668 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:49, loss = 0.646950, train_acc = 0.668 
07-18 01:43 [MainProcess, 10268] [INFO ]  Iteration:49, loss = 0.646950, train_acc = 0.668 


In [135]:
audio_feature, age, sex, hw, preg, loc, data_label_torch = next(iter(dataloader_train_murmurs))

NameError: name 'dataloader_train_murmurs' is not defined

In [112]:
audio_feature.shape

torch.Size([16, 115136])

In [133]:
len(dataloader_train_murmurs)

1

In [236]:
nnet

vgg_hj(
  (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1

In [235]:
nnet = vgg_hj(num_murmur_classes)

In [44]:
#imputer = SimpleImputer().fit(features['mel1'])
#features['mel1'] = imputer.transform(features['mel1'])
log_dir = './log/'
project = 'hj0'
logger = get_logger(log_dir + '/' + project)

# Define model
nnet = vgg_hj(num_murmur_classes) 
nnet = nnet.cuda()
optimizer = optim.Adam(nnet.parameters(), lr=4e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.91)
criterion = nn.CrossEntropyLoss()
    

for iter_ in range(50):  # args.end_iter
    start_time = time.time()
    running_loss = 0.0
    train_epoch_acc = 0.0
    train_acc = 0.0
    nnet.train()
    labels=[]
    idx = 0
#    epoch[str(iter_)] = {}
    for audio_feature, data_label_torch in tqdm(dataloader_train):  
            #print(audio_feature.shape)
        
        #idx = idx + 1
        #audio_feature = torch.nan_to_num(audio_feature, nan=0)
        audio_feature[audio_feature != audio_feature] = 0
        
        #audio_feature = audio_feature.permute(1,3,0,2)
        #print(audio_feature.shape)
        audio_feature = audio_feature.cuda()
        #age = age.permute(1,0).float().cuda()
            #print(age.shape)
        #    sex = sex.permute(1,0).float().cuda()
            #print(sex.shape)
        #    hw = hw.permute(1,0).float().cuda()
            #print(hw.shape)
        #    preg = preg.permute(1,0).float().cuda()
            #print(preg.shape)
         #   loc = loc.permute(1,0).float().cuda()
            #print(loc.shape)
    #        for idx in range(len(data_label_torch)) :
    #            data_label_torch[idx][0] = mapping[int(data_label_torch[idx][0])]
        data_label_torch = data_label_torch.cuda()
                
        data_label_torch = data_label_torch.long()
        outputs = nnet(input=audio_feature)
            
        #softmax = nn.Softmax(dim = 1)
            
        #outputs = softmax(outputs)
            
        #labels = np.argmax(outputs.detach().cpu().numpy())
        murmur_labels = np.zeros((audio_feature.shape[0],len(murmur_classes)), dtype=np.int_)
        for i in range(audio_feature.shape[0]):
            idx = np.argmax(outputs[i].detach().cpu().numpy())
            murmur_labels[i][idx] = 1 
            
            
        labels = torch.argmax(data_label_torch, 1)    
            
        loss = criterion(outputs.float(),labels)
        #train_acc = compute_weighted_accuracy(data_label_torch.squeeze(1), outputs,murmur_classes )
        #print(outputs.shape)
        #print(murmur_labels.shape)
            
        train_acc = compute_weighted_accuracy(murmur_labels, data_label_torch.detach().cpu().numpy(),murmur_classes)
        #print(train_acc)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 2.5)
        optimizer.step()

        running_loss += loss.item()
        train_epoch_acc += train_acc.item()
            #if idx % 100 == 0 :
            #    print("lr: ", optimizer.param_groups[0]['lr'])
            #    print('batch : ',idx, ' done')
    logging.info("Iteration:{0}, loss = {1:.6f}, train_acc = {2:.3f} ".format(iter_, running_loss/all_file_train, train_epoch_acc/all_file_train))
    
save_challenge_model(model_folder, murmur_classes, nnet)
                

100%|██████████| 159/159 [00:02<00:00, 65.12it/s]

07-20 15:10 [MainProcess, 32409] [INFO ]  Iteration:0, loss = 0.773684, train_acc = 0.655 
07-20 15:10 [MainProcess, 32409] [INFO ]  Iteration:0, loss = 0.773684, train_acc = 0.655 
07-20 15:10 [MainProcess, 32409] [INFO ]  Iteration:0, loss = 0.773684, train_acc = 0.655 
07-20 15:10 [MainProcess, 32409] [INFO ]  Iteration:0, loss = 0.773684, train_acc = 0.655 
07-20 15:10 [MainProcess, 32409] [INFO ]  Iteration:0, loss = 0.773684, train_acc = 0.655 



100%|██████████| 159/159 [00:02<00:00, 63.96it/s]

07-20 15:10 [MainProcess, 32409] [INFO ]  Iteration:1, loss = 0.773863, train_acc = 0.675 
07-20 15:10 [MainProcess, 32409] [INFO ]  Iteration:1, loss = 0.773863, train_acc = 0.675 
07-20 15:10 [MainProcess, 32409] [INFO ]  Iteration:1, loss = 0.773863, train_acc = 0.675 
07-20 15:10 [MainProcess, 32409] [INFO ]  Iteration:1, loss = 0.773863, train_acc = 0.675 
07-20 15:10 [MainProcess, 32409] [INFO ]  Iteration:1, loss = 0.773863, train_acc = 0.675 



100%|██████████| 159/159 [00:02<00:00, 62.92it/s]

...

07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:47, loss = 0.610553, train_acc = 0.755 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:47, loss = 0.610553, train_acc = 0.755 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:47, loss = 0.610553, train_acc = 0.755 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:47, loss = 0.610553, train_acc = 0.755 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:47, loss = 0.610553, train_acc = 0.755 



100%|██████████| 159/159 [00:02<00:00, 62.92it/s]

07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:48, loss = 0.605963, train_acc = 0.750 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:48, loss = 0.605963, train_acc = 0.750 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:48, loss = 0.605963, train_acc = 0.750 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:48, loss = 0.605963, train_acc = 0.750 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:48, loss = 0.605963, train_acc = 0.750 



100%|██████████| 159/159 [00:02<00:00, 64.29it/s]

07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:49, loss = 0.600134, train_acc = 0.760 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:49, loss = 0.600134, train_acc = 0.760 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:49, loss = 0.600134, train_acc = 0.760 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:49, loss = 0.600134, train_acc = 0.760 
07-20 15:12 [MainProcess, 32409] [INFO ]  Iteration:49, loss = 0.600134, train_acc = 0.760 


In [60]:
model_folder

'./model'

In [61]:
nnet

vgg_hj(
  (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1

In [228]:
!nvidia-smi

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

Wed Jul 20 14:26:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 44%   48C   

In [152]:
nnet

vgg_hj(
  (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1

In [43]:
features[0]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

## 모델 저장

In [119]:
trained_model =  torch.load('./model/model.sav')

RuntimeError: Invalid magic number; corrupt file?

In [120]:
nnet

vgg_hj(
  (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [95]:
features['mel1']

array([array([[-0.19912574,  0.5145373 ,  0.1508112 , ..., -0.46197197,
                0.04964472,  0.14100385],
              [-0.22853336,  0.34318015,  0.12647286, ..., -0.1666474 ,
                0.2702695 ,  0.10539503],
              [-0.08499706,  0.1853809 ,  0.19848159, ..., -0.20012715,
                0.06223098,  0.15367241],
              ...,
              [-0.1789976 ,  0.50544095,  0.12959465, ..., -0.17574063,
                0.20017333, -0.00640017],
              [-0.2743313 ,  0.41213012,  0.20129842, ..., -0.22131136,
                0.12829405,  0.0321559 ],
              [-0.30686334,  0.18500513,  0.2467196 , ..., -0.37595025,
                0.17643583, -0.0107069 ]], dtype=float32)              ,
       array([[-0.32665142,  0.7796868 ,  0.05629933, ..., -0.47412428,
               -0.00358604,  0.07075846],
              [-0.36447236,  0.56911767, -0.04721925, ..., -0.45891872,
                0.10469179, -0.00582892],
              [-0.4747667 ,  0.2953822

In [93]:
features['mel1'] = np.array(features['mel1'])

In [94]:
audio_feature = torch.from_numpy(features['mel1'])

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [117]:
features['mel1'][3].shape

(15, 128)

In [100]:
float_arr = np.vstack(features['mel1']).astype(np.float64)

In [122]:
torch.from_numpy(float_arr).shape


torch.Size([65, 128])

## run model

In [46]:
def run_challenge_model(model, data, recordings, verbose):
    
#    if model['model'] == 'toy_ti' :
#        model1 = get_toy(model['mel_shape'])
#    model1.load_weights(model['model_fnm'])
    GPU_NUM = 0 # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
#    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device)
    
    #patient_files = find_patient_files(data)
    #num_patient_files = len(patient_files)

    #if model['murmur_classifier'] == 'vgg_hj' :
    trained_model =  torch.load('./model/hjvgg_model.hdf5')
    nnet = vgg_hj(3)
    nnet.load_state_dict(trained_model)
    nnet = nnet.cuda()
    #data_folder = '/Data2/hmd/data_split/murmur/test'
    nnet.eval()
    
    classes = classes = ['Present', 'Unknown', 'Absent']

    features['mel1'] = []

    num_locations = get_num_locations(data)
    
    recording_information = data.split('\n')[1:num_locations+1]

 
    for j in range(num_locations) :
        entries = recording_information[j].split(' ')
        recording_file = entries[2]#음성기록 
        filename = os.path.join(data_folder, recording_file)
        mel1 = vgg_feature(filename)
        features['mel1'].append(mel1)

    features['mel1'] = np.array(features['mel1'])
    
    float_arr = np.vstack(features['mel1']).astype(np.float64)
    
    
    audio_feature = torch.from_numpy(float_arr)

    audio_feature = audio_feature.cuda()
    
    audio_feature = audio_feature.unsqueeze(0)
    
    
    outputs = nnet(input=audio_feature)
    softmax = nn.Softmax(dim = 1)
    outputs = softmax(outputs)
    
    prob1 = outputs.mean(axis = 0)
    
    prob1 = prob1.data.cpu().numpy()
    
    
    
    
    
#    print(prob1.shape)
    
#    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])

    # Get classifier probabilities.
#    prob1 = res1.mean(axis = 0) ## simple rule for now
    idx = np.argmax(prob1)
    # Choose label with higher probability.
    labels = np.zeros(len(classes), dtype=np.int_)
    labels[idx] = 1

    return classes, labels, prob1

In [ ]:
hmd/murmur

In [48]:
nnet = vgg_hj(3)

In [49]:
nnet

vgg_hj(
  (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1

In [50]:
trained_model =  torch.load('./model/hjvgg_model.hdf5')

In [54]:
vgg_hj(3)

vgg_hj(
  (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1

In [51]:
trained_model

OrderedDict([('features.0.weight',
              tensor([[[[-2.9218e-01,  2.4075e-01, -2.7257e-01],
                        [ 1.5789e-01, -6.6873e-03, -2.6994e-01],
                        [-1.9574e-01, -2.8377e-01,  9.5330e-02]]],
              
              
                      [[[-2.0406e-01, -1.0313e-01,  2.1278e-01],
                        [ 1.3091e-01,  3.2212e-01,  8.0697e-02],
                        [-1.0597e-01, -2.1543e-01, -1.4974e-01]]],
              
              
                      [[[ 3.2943e-02,  3.9747e-02, -9.6264e-03],
                        [-2.3437e-02, -1.7899e-01,  1.4243e-01],
                        [ 1.4712e-01,  1.6075e-01, -5.8887e-03]]],
              
              
                      [[[-3.2819e-01,  2.7474e-01, -5.1563e-02],
                        [ 7.6252e-02, -1.7158e-01, -2.6543e-01],
                        [-7.2321e-02,  1.3667e-01, -1.0971e-01]]],
              
              
                      [[[ 6.9053e-02, -1.6973e-01, -3.485

In [59]:
trained_model

OrderedDict([('features.0.weight',
              tensor([[[[-2.9218e-01,  2.4075e-01, -2.7257e-01],
                        [ 1.5789e-01, -6.6873e-03, -2.6994e-01],
                        [-1.9574e-01, -2.8377e-01,  9.5330e-02]]],
              
              
                      [[[-2.0406e-01, -1.0313e-01,  2.1278e-01],
                        [ 1.3091e-01,  3.2212e-01,  8.0697e-02],
                        [-1.0597e-01, -2.1543e-01, -1.4974e-01]]],
              
              
                      [[[ 3.2943e-02,  3.9747e-02, -9.6264e-03],
                        [-2.3437e-02, -1.7899e-01,  1.4243e-01],
                        [ 1.4712e-01,  1.6075e-01, -5.8887e-03]]],
              
              
                      [[[-3.2819e-01,  2.7474e-01, -5.1563e-02],
                        [ 7.6252e-02, -1.7158e-01, -2.6543e-01],
                        [-7.2321e-02,  1.3667e-01, -1.0971e-01]]],
              
              
                      [[[ 6.9053e-02, -1.6973e-01, -3.485

In [47]:
trained_model =  torch.load('./model/hjvgg_model.hdf5')
nnet = vgg_hj(3)
nnet.load_state_dict(trained_model)
nnet = nnet.cuda()
data_folder = '/home/hj20/hmd/murmur/test'
nnet.eval()

RuntimeError: Error(s) in loading state_dict for vgg_hj:
	Missing key(s) in state_dict: "bn0.weight", "bn0.bias", "bn0.running_mean", "bn0.running_var", "conv_block1.conv1.weight", "conv_block1.conv2.weight", "conv_block1.bn1.weight", "conv_block1.bn1.bias", "conv_block1.bn1.running_mean", "conv_block1.bn1.running_var", "conv_block1.bn2.weight", "conv_block1.bn2.bias", "conv_block1.bn2.running_mean", "conv_block1.bn2.running_var", "conv_block2.conv1.weight", "conv_block2.conv2.weight", "conv_block2.bn1.weight", "conv_block2.bn1.bias", "conv_block2.bn1.running_mean", "conv_block2.bn1.running_var", "conv_block2.bn2.weight", "conv_block2.bn2.bias", "conv_block2.bn2.running_mean", "conv_block2.bn2.running_var", "conv_block3.conv1.weight", "conv_block3.conv2.weight", "conv_block3.bn1.weight", "conv_block3.bn1.bias", "conv_block3.bn1.running_mean", "conv_block3.bn1.running_var", "conv_block3.bn2.weight", "conv_block3.bn2.bias", "conv_block3.bn2.running_mean", "conv_block3.bn2.running_var", "

In [45]:
model_folder = "./model"
output_folder = "./test_output"
allow_failures = False

In [77]:
patient_files = find_patient_files(data_folder)
num_patient_files = len(patient_files)

In [78]:
if num_patient_files==0:
    raise Exception('No data was provided.')

# Create a folder for the Challenge outputs if it does not already exist.
os.makedirs(output_folder, exist_ok=True)

In [83]:
verbose =1

In [139]:
# Iterate over the patient files.
for i in range(num_patient_files):
    patient_data = load_patient_data(patient_files[i])
    recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
    try:
        classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
    except:
        if allow_failures:
            if verbose >= 2:
                print('... failed.')
            classes, labels, probabilities = list(), list(), list()
        else:
            raise

    # Save Challenge outputs.
    head, tail = os.path.split(patient_files[i])
    root, extension = os.path.splitext(tail)
    output_file = os.path.join(output_folder, root + '.csv')
    patient_id = get_patient_id(patient_data)
    save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

print('Done.')

/home/hj20/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Done.


In [ ]:
#!/usr/bin/env python

# Do *not* edit this script. Changes will be discarded so that we can process the models consistently.

# This file contains functions for running models for the 2022 Challenge. You can run it as follows:
#
#   python run_model.py model data outputs
#
# where 'model' is a folder containing the your trained model, 'data' is a folder containing the Challenge data, and 'outputs' is a
# folder for saving your model's outputs.

import numpy as np, os, sys
from helper_code import *
from team_code import load_challenge_model, run_challenge_model

# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!
    
    
    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

if __name__ == '__main__':
    # Parse the arguments.
    if not (len(sys.argv) == 4 or len(sys.argv) == 5):
        raise Exception('Include the model, data, and output folders as arguments, e.g., python run_model.py model data outputs.')

    # Define the model, data, and output folders.
    model_folder = sys.argv[1]
    data_folder = sys.argv[2]
    output_folder = sys.argv[3]

    # Allow or disallow the model to fail on parts of the data; helpful for debugging.
    allow_failures = False

    # Change the level of verbosity; helpful for debugging.
    if len(sys.argv)==5 and is_integer(sys.argv[4]):
        verbose = int(sys.argv[4])
    else:
        verbose = 1

    run_model(model_folder, data_folder, output_folder, allow_failures, verbose)


In [ ]:
def run_challenge_model(model, data, recordings, verbose):
    murmur_classes = model['murmur_classes']
    murmur_classifier = model['murmur_classifier']
    outcome_classes = model['outcome_classes']
    outcome_classifier = model['outcome_classifier']

    # Load features.
    features = get_features_vgg(data, recordings)
    
    # Impute missing data.
    #features = features.reshape(1, -1)
    #features = imputer.transform(features)

    # Get classifier probabilities.
    murmur_probabilities = murmur_classifier(features)
    murmur_probabilities = np.asarray(murmur_probabilities, dtype=np.float32)[:, 0, 1]
    outcome_probabilities = outcome_classifier.predict_proba(features)
    outcome_probabilities = np.asarray(outcome_probabilities, dtype=np.float32)[:, 0, 1]

    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1

    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

    return classes, labels, probabilities

In [ ]:
murmur_classifier(feature)

In [62]:
# Save Challenge outputs.
def save_challenge_outputs(filename, patient_id, classes, labels, probabilities):
    # Format Challenge outputs.
    patient_string = '#{}'.format(patient_id)
    class_string = ','.join(str(c) for c in classes)
    label_string = ','.join(str(l) for l in labels)
    probabilities_string = ','.join(str(p) for p in probabilities)
    output_string = patient_string + '\n' + class_string + '\n' + label_string + '\n' + probabilities_string + '\n'

    # Write the Challenge outputs.
    with open(filename, 'w') as f:
        f.write(output_string)

# Load Challenge outputs.
def load_challenge_outputs(filename):
    with open(filename, 'r') as f:
        for i, l in enumerate(f):
            if i==0:
                patient_id = l.replace('#', '').strip()
            elif i==1:
                classes = tuple(entry.strip() for entry in l.split(','))
            elif i==2:
                labels = tuple(sanitize_binary_value(entry) for entry in l.split(','))
            elif i==3:
                probabilities = tuple(sanitize_scalar_value(entry) for entry in l.split(','))
            else:
                break
    return patient_id, classes, labels, probabilities

In [ ]:
#!/usr/bin/env python

# Do *not* edit this script. Changes will be discarded so that we can process the models consistently.

# This file contains functions for running models for the 2022 Challenge. You can run it as follows:
#
#   python run_model.py model data outputs
#
# where 'model' is a folder containing the your trained model, 'data' is a folder containing the Challenge data, and 'outputs' is a
# folder for saving your model's outputs.

import numpy as np, os, sys
from helper_code import *
from team_code import load_challenge_model, run_challenge_model

# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!
    
    
    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

if __name__ == '__main__':
    # Parse the arguments.
    if not (len(sys.argv) == 4 or len(sys.argv) == 5):
        raise Exception('Include the model, data, and output folders as arguments, e.g., python run_model.py model data outputs.')

    # Define the model, data, and output folders.
    model_folder = sys.argv[1]
    data_folder = sys.argv[2]
    output_folder = sys.argv[3]

    # Allow or disallow the model to fail on parts of the data; helpful for debugging.
    allow_failures = False

    # Change the level of verbosity; helpful for debugging.
    if len(sys.argv)==5 and is_integer(sys.argv[4]):
        verbose = int(sys.argv[4])
    else:
        verbose = 1

    run_model(model_folder, data_folder, output_folder, allow_failures, verbose)


In [103]:
data_label_torch

tensor([0, 2, 2, 1, 0, 2, 2, 1, 2, 1, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 1, 0, 0,
        0, 2, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 2, 0], device='cuda:0')

In [102]:
murmur_labels

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0,

In [109]:
audio_feature, age, sex, hw, preg, loc, data_label_torch = next(iter(dataloader_train_murmurs))

In [116]:
#def save_challenge_model(model_folder, murmur_classes, murmur_classifier):
#    d = {'murmur_classes': murmur_classes, 'murmur_classifier': murmur_classifier}
#    filename = os.path.join(model_folder, 'model.sav')
#    joblib.dump(d, filename, protocol=0)

In [43]:
def save_challenge_model(model_folder,murmur_classes, murmur_classifier):
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename = os.path.join(model_folder, 'hj_vgg' + '_model.hdf5')
    torch.save(model.state_dict(), filename)
    d = {'murmur_classes': murmur_classes, 'murmur_classifier': murmur_classifier}
    with open(info_fnm, 'wb') as f:
        pk.dump(d, f, pk.HIGHEST_PROTOCOL)

In [111]:
data_label_torch.shape

torch.Size([64, 3])

In [102]:
outputs

tensor([[1.6035e-07, 5.1546e-08, 1.0000e+00],
        [9.9909e-12, 9.9395e-12, 1.0000e+00],
        [8.5100e-13, 3.0245e-13, 1.0000e+00],
        [4.3278e-09, 1.3088e-09, 1.0000e+00],
        [3.3205e-09, 3.7477e-08, 1.0000e+00],
        [4.2638e-08, 3.0313e-08, 1.0000e+00],
        [2.2081e-09, 2.1774e-09, 1.0000e+00],
        [1.3217e-09, 1.0547e-08, 1.0000e+00],
        [1.6628e-09, 1.0898e-07, 1.0000e+00],
        [2.8049e-11, 6.2666e-09, 1.0000e+00],
        [6.0245e-10, 8.7798e-11, 1.0000e+00],
        [1.7271e-06, 3.9729e-08, 1.0000e+00],
        [6.9074e-09, 3.9334e-11, 1.0000e+00],
        [2.2225e-14, 3.5935e-13, 1.0000e+00],
        [8.9302e-10, 4.1291e-10, 1.0000e+00],
        [1.7465e-07, 4.6608e-06, 1.0000e+00],
        [4.8258e-08, 2.2146e-07, 1.0000e+00],
        [6.0823e-08, 3.4511e-09, 1.0000e+00],
        [1.1943e-09, 1.0225e-09, 1.0000e+00],
        [6.4007e-09, 2.2116e-09, 1.0000e+00],
        [7.1826e-07, 1.6643e-07, 1.0000e+00],
        [6.4189e-10, 1.7071e-09, 1

In [100]:
labels = torch.argmax(data_label_torch, 1) 

In [99]:
sum(outputs[0])

tensor(1.0000, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
#기존 코드 
def save_challenge_model(model_folder, imputer, murmur_classes, murmur_classifier, outcome_classes, outcome_classifier):
    d = {'imputer': imputer, 'murmur_classes': murmur_classes, 'murmur_classifier': murmur_classifier, 'outcome_classes': outcome_classes, 'outcome_classifier': outcome_classifier}
    filename = os.path.join(model_folder, 'model.sav')
    joblib.dump(d, filename, protocol=0)

In [69]:
data_label_torch

tensor([0, 2, 2, 1, 0, 2, 2, 1, 2, 1, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 1, 0, 0,
        0, 2, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 2, 0], device='cuda:0')

In [80]:
import logging

In [79]:
all_file_train

40

In [68]:
murmur_labels

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1,

In [65]:
train_acc = compute_weighted_accuracy(outputs.detach().cpu().numpy(), murmur_labels,murmur_classes)

AssertionError: 

In [58]:
data_label_torch

tensor([[1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 

In [59]:
torch.argmax(data_label_torch, 1)

tensor([0, 2, 2, 1, 0, 2, 2, 1, 2, 1, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 1, 0, 0,
        0, 2, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 2, 0], device='cuda:0')

In [162]:
data_label_torch.detach().cpu().numpy()[0]

array([1, 0, 0])

In [176]:
murmur_labels = np.zeros((64,len(murmur_classes)), dtype=np.int_)

In [179]:
murmur_labels

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0,

In [40]:
 softmax = nn.Softmax(dim = 1)

In [53]:
outputs = softmax(outputs)
            
#labels = np.argmax(outputs.detach().cpu().numpy())
murmur_labels = np.zeros((64,len(murmur_classes)), dtype=np.int_)
for i in range(64):
    idx = np.argmax(outputs[i].detach().cpu().numpy())
    murmur_labels[i][idx] = 1 

In [50]:
np.argmax(outputs[0].detach().cpu().numpy())

2

In [51]:
murmur_labels[idx] = 1

In [49]:
murmur_labels = np.zeros((64,len(murmur_classes)), dtype=np.int_)

In [61]:
audio_feature.shape

torch.Size([64, 34, 128])

In [54]:
murmur_labels

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0,

In [138]:
softmax = nn.Softmax(dim = 1)
            
outputs = softmax(outputs)
            
labels = np.argmax(outputs.detach().cpu().numpy())
        

In [143]:
outputs[0]

tensor([0.5060, 0.0382, 0.4558], device='cuda:0', grad_fn=<SelectBackward>)

In [147]:
murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
idx = np.argmax(outputs.detach().cpu().numpy())
murmur_labels[idx] = 1

In [144]:
murmur_classes

['Present', 'Unknown', 'Absent']

In [148]:
murmur_labels

array([1, 0, 0])

In [ ]:
# Load murmurs from label files.
def load_murmurs(label_files, classes):
    num_patients = len(label_files)
    num_classes = len(classes)

    # Use one-hot encoding for the labels.
    labels = np.zeros((num_patients, num_classes), dtype=np.bool_)

    # Iterate over the patients.
    for i in range(num_patients):
        data = load_patient_data(label_files[i])
        label = get_murmur(data)
        for j, x in enumerate(classes):
            if compare_strings(label, x):
                labels[i, j] = 1

In [77]:
data_label_torch

tensor([[[1],
         [0],
         [0]],

        [[0],
         [0],
         [1]],

        [[0],
         [0],
         [1]],

        [[0],
         [1],
         [0]],

        [[1],
         [0],
         [0]],

        [[0],
         [0],
         [1]],

        [[0],
         [0],
         [1]],

        [[0],
         [1],
         [0]],

        [[0],
         [0],
         [1]],

        [[0],
         [1],
         [0]],

        [[0],
         [0],
         [1]],

        [[1],
         [0],
         [0]],

        [[0],
         [0],
         [1]],

        [[0],
         [0],
         [1]],

        [[1],
         [0],
         [0]],

        [[1],
         [0],
         [0]],

        [[1],
         [0],
         [0]],

        [[1],
         [0],
         [0]],

        [[0],
         [0],
         [1]],

        [[0],
         [0],
         [1]],

        [[1],
         [0],
         [0]],

        [[0],
         [1],
         [0]],

        [[1],
         [0],
    

In [126]:
all_file_train = len(dataloader_train)

In [23]:
nnet

vgg_hj(
  (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [31]:
audio_feature.shape

NameError: name 'audio_feature' is not defined

In [39]:
audio_feature = audio_feature.unsqueeze(1)

In [59]:
audio_feature=audio_feature.cuda()

RuntimeError: CUDA error: device-side assert triggered

In [129]:
nnet(audio_feature)[0].shape

torch.Size([64, 1, 34, 128])
tensor([[[[5.7412e-01, 0.0000e+00, 3.3869e-01,  ..., 1.2520e-01,
           7.7155e-01, 2.6079e-01],
          [6.8532e-01, 1.0659e+00, 8.5922e-01,  ..., 1.9839e-01,
           4.5233e-01, 3.5609e-01],
          [5.0707e-01, 2.3067e-01, 6.3375e-01,  ..., 2.7780e-01,
           3.6742e-01, 0.0000e+00],
          ...,
          [1.0564e+00, 3.0016e-01, 2.9767e-01,  ..., 2.1634e-01,
           1.2926e+00, 4.7758e-01],
          [0.0000e+00, 0.0000e+00, 2.7249e-01,  ..., 0.0000e+00,
           7.8129e-01, 0.0000e+00],
          [2.4336e-01, 8.9684e-01, 0.0000e+00,  ..., 7.1450e-01,
           6.5980e-02, 2.2590e-01]],

         [[0.0000e+00, 6.5706e-01, 0.0000e+00,  ..., 0.0000e+00,
           4.9547e-02, 1.5031e-01],
          [2.8260e-01, 6.4526e-01, 6.0107e-01,  ..., 1.1780e+00,
           1.0454e+00, 4.4238e-01],
          [5.9264e-01, 0.0000e+00, 9.0780e-01,  ..., 6.1395e-01,
           5.9984e-01, 1.9291e-01],
          ...,
          [2.5893e-01, 0.0000e

torch.Size([3])

In [55]:
murmurs.shape

(2532, 3)

In [ ]:
import sys
from helper_code import is_integer
#from team_code import train_challenge_model

if __name__ == '__main__':
    # Parse the arguments.
    if not (len(sys.argv) == 3 or len(sys.argv) == 4):
        raise Exception('Include the data and model folders as arguments, e.g., python train_model.py data model.')

    # Define the data and model foldes.
    train_folder = '/Data2/hmd/data_split/murmur/train'
    #test_folder ='/Data2/hmd/data_split/murmur/test'
    model_folder = 'model'

    # Change the level of verbosity; helpful for debugging.
    if len(sys.argv)==4 and is_integer(sys.argv[3]):
        verbose = int(sys.argv[3])
    else:
        verbose = 1

        
        
        
    train_challenge_model(train_folder, model_folder, verbose) ### Teams: Implement this function!!!

In [ ]:
return features, murmurs, outcomes

In [20]:
#모델 폴더 생성
os.makedirs(model_folder, exist_ok=True)

In [36]:
murmur_classes = ['Present', 'Unknown', 'Absent']
num_murmur_classes = len(murmur_classes)
outcome_classes = ['Abnormal', 'Normal']
num_outcome_classes = len(outcome_classes)

In [23]:
features = list()
murmurs = list()
outcomes = list()

In [189]:
for i in range(num_patient_files):
    #if verbose >= 2:
        #print('    {}/{}...'.format(i+1, num_patient_files))

    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    current_recordings = load_recordings(data_folder, current_patient_data
    features.append(current_features)                                     

In [194]:
current_recordings[0].shape

(80000,)

In [25]:
#기본 베이스라인 피쳐 
for i in range(num_patient_files):
    #if verbose >= 2:
        #print('    {}/{}...'.format(i+1, num_patient_files))

    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    current_recordings = load_recordings(data_folder, current_patient_data)

    # Extract features
    current_features = get_features(current_patient_data, current_recordings)
    features.append(current_features)

    # Extract labels and use one-hot encoding.
    current_murmur = np.zeros(num_murmur_classes, dtype=int)
    murmur = get_murmur(current_patient_data)
    if murmur in murmur_classes:
        j = murmur_classes.index(murmur)
        current_murmur[j] = 1
    murmurs.append(current_murmur)

    current_outcome = np.zeros(num_outcome_classes, dtype=int)
    outcome = get_outcome(current_patient_data)
    if outcome in outcome_classes:
        j = outcome_classes.index(outcome)
        current_outcome[j] = 1
    outcomes.append(current_outcome)

features = np.vstack(features)
murmurs = np.vstack(murmurs)
outcomes = np.vstack(outcomes)

In [38]:
from tqdm import tqdm

In [41]:
patient_files[0]

'/home/hj20/hmd/murmur/train/2530.txt'

In [42]:
current_patient_data = load_patient_data(patient_files[0])
current_recordings = load_recordings(data_folder, current_patient_data)

In [43]:
current_recordings

[array([-425, 1045,  518, ...,  509,  443,  122], dtype=int16),
 array([2593, 1874, 1370, ..., -637, -255, -243], dtype=int16),
 array([ 2276,  2343,  2448, ..., -2732, -5634, -4290], dtype=int16),
 array([12672, 10041,  3215, ...,  -658,     8,   528], dtype=int16)]

In [44]:
current_patient_data

'2530 4 4000\nAV 2530_AV.hea 2530_AV.wav 2530_AV.tsv\nPV 2530_PV.hea 2530_PV.wav 2530_PV.tsv\nTV 2530_TV.hea 2530_TV.wav 2530_TV.tsv\nMV 2530_MV.hea 2530_MV.wav 2530_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 98.0\n#Weight: 15.9\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [49]:
#vgg피쳐 
for i in range(num_patient_files):
    #if verbose >= 2:
        #print('    {}/{}...'.format(i+1, num_patient_files))

    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    current_recordings = load_recordings(data_folder, current_patient_data)

    # Extract features
    #current_features = get_features_vgg(current_patient_data, current_recordings)
    #features.append(current_features)

    # Extract labels and use one-hot encoding.
    current_murmur = np.zeros(num_murmur_classes, dtype=int)
    murmur = get_murmur(current_patient_data)
    if murmur in murmur_classes:
        j = murmur_classes.index(murmur)
        current_murmur[j] = 1
    murmurs.append(current_murmur)

    current_outcome = np.zeros(num_outcome_classes, dtype=int)
    outcome = get_outcome(current_patient_data)
    if outcome in outcome_classes:
        j = outcome_classes.index(outcome)
        current_outcome[j] = 1
    outcomes.append(current_outcome)

features = np.vstack(features)
murmurs = np.vstack(murmurs)
outcomes = np.vstack(outcomes)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [27]:
features.shape

(751, 26)

In [28]:
murmurs.shape

(751, 3)

In [30]:
murmurs #원핫인코딩 

array([[0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       ...,
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1]])

In [31]:
outcomes#바이너리 확인

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [0, 1]])

In [32]:
imputer = SimpleImputer().fit(features)
features = imputer.transform(features)

In [ ]:
murmur_classifier = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, murmurs)
outcome_classifier = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, outcomes)

In [ ]:
#함수짜기
def murmur_classifier()

In [2]:
#!/usr/bin/env python

# Edit this script to add your team's code. Some functions are *required*, but you can edit most parts of the required functions,
# change or remove non-required functions, and add your own functions.

################################################################################
#
# Import libraries and functions. You can change or remove them.
#
################################################################################

from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

################################################################################
#
# Required functions. Edit these functions to add your code, but do not change the arguments.
#
################################################################################

# Train your model.
def train_challenge_model(data_folder, model_folder, verbose):
    # Find data files.
    if verbose >= 1:
        print('Finding data files...')

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the model if it does not already exist.
    os.makedirs(model_folder, exist_ok=True)

    # Extract the features and labels.
    if verbose >= 1:
        print('Extracting features and labels from the Challenge data...')

    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)

    features = list()
    murmurs = list()
    outcomes = list()

    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files[i])
        current_recordings = load_recordings(data_folder, current_patient_data)

        # Extract features
        current_features = get_features(current_patient_data, current_recordings)
        features.append(current_features)

        # Extract labels and use one-hot encoding.
        current_murmur = np.zeros(num_murmur_classes, dtype=int)
        murmur = get_murmur(current_patient_data)
        if murmur in murmur_classes:
            j = murmur_classes.index(murmur)
            current_murmur[j] = 1
        murmurs.append(current_murmur)

        current_outcome = np.zeros(num_outcome_classes, dtype=int)
        outcome = get_outcome(current_patient_data)
        if outcome in outcome_classes:
            j = outcome_classes.index(outcome)
            current_outcome[j] = 1
        outcomes.append(current_outcome)

    features = np.vstack(features)
    murmurs = np.vstack(murmurs)
    outcomes = np.vstack(outcomes)

    # Train the model.
    if verbose >= 1:
        print('Training model...')

    # Define parameters for random forest classifier.
    #n_estimators   = 123  # Number of trees in the forest.
    #max_leaf_nodes = 45   # Maximum number of leaf nodes in each tree.
    #random_state   = 6789 # Random state; set for reproducibility.

    imputer = SimpleImputer().fit(features)
    features = imputer.transform(features)
    #학습코드 여기 수정해야함 
    murmur_classifier = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, murmurs)
    outcome_classifier = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, outcomes)

    # Save the model.
    save_challenge_model(model_folder, imputer, murmur_classes, murmur_classifier, outcome_classes, outcome_classifier)

    if verbose >= 1:
        print('Done.')

# Load your trained model. This function is *required*. You should edit this function to add your code, but do *not* change the
# arguments of this function.
def load_challenge_model(model_folder, verbose):
    filename = os.path.join(model_folder, 'model.sav')
    return joblib.load(filename)

# Run your trained model. This function is *required*. You should edit this function to add your code, but do *not* change the
# arguments of this function.
def run_challenge_model(model, data, recordings, verbose):
    imputer = model['imputer']
    murmur_classes = model['murmur_classes']
    murmur_classifier = model['murmur_classifier']
    outcome_classes = model['outcome_classes']
    outcome_classifier = model['outcome_classifier']

    # Load features.
    features = get_features(data, recordings)

    # Impute missing data.
    features = features.reshape(1, -1)
    features = imputer.transform(features)

    # Get classifier probabilities.
    murmur_probabilities = murmur_classifier.predict_proba(features)
    murmur_probabilities = np.asarray(murmur_probabilities, dtype=np.float32)[:, 0, 1]
    outcome_probabilities = outcome_classifier.predict_proba(features)
    outcome_probabilities = np.asarray(outcome_probabilities, dtype=np.float32)[:, 0, 1]

    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1

    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

    return classes, labels, probabilities

################################################################################
#
# Optional functions. You can change or remove these functions and/or add new functions.
#
################################################################################

# Save your trained model.
def save_challenge_model(model_folder, imputer, murmur_classes, murmur_classifier, outcome_classes, outcome_classifier):
    d = {'imputer': imputer, 'murmur_classes': murmur_classes, 'murmur_classifier': murmur_classifier, 'outcome_classes': outcome_classes, 'outcome_classifier': outcome_classifier}
    filename = os.path.join(model_folder, 'model.sav')
    joblib.dump(d, filename, protocol=0)



In [57]:
X = np.array(features,dtype=object)

In [69]:
X[3].shape

(27, 128)

In [70]:
features

[array([[-0.34538344,  0.13368073,  0.12524429, ..., -0.31086156,
          0.14979401, -0.48415416],
        [-0.19154432,  0.214212  ,  0.28560516, ..., -0.39440694,
         -0.11627233, -0.14756796],
        [-0.24413106,  0.3157343 ,  0.06686218, ..., -0.28411326,
         -0.06426099, -0.37615448],
        ...,
        [-0.24816701,  0.02995777,  0.23249248, ..., -0.5202584 ,
         -0.12005882, -0.25576204],
        [-0.35670713, -0.07676217, -0.10251346, ..., -0.643227  ,
         -0.00666235, -0.71922445],
        [-0.06675744,  0.03318772,  0.05104068, ..., -0.22440037,
         -0.0809126 , -0.2656672 ]], dtype=float32),
 array([[-0.36357376, -0.11491951,  0.47113556, ..., -0.64488626,
         -0.16923968, -0.21453565],
        [-0.25615296,  0.09203072,  0.26022577, ..., -0.42452005,
         -0.03151461, -0.01584616],
        [-0.14047906,  0.16528079,  0.23086849, ..., -0.2352452 ,
          0.07583477,  0.18939807],
        ...,
        [-0.3527775 , -0.2846978 ,  0.1

In [101]:
murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')

NameError: name 'murmur_labels' is not defined

3072

In [24]:
# Extract features from the data.
def get_features(data, recordings):
    # Extract the age group and replace with the (approximate) number of months for the middle of the age group.
    age_group = get_age(data)

    if compare_strings(age_group, 'Neonate'):
        age = 0.5
    elif compare_strings(age_group, 'Infant'):
        age = 6
    elif compare_strings(age_group, 'Child'):
        age = 6 * 12
    elif compare_strings(age_group, 'Adolescent'):
        age = 15 * 12
    elif compare_strings(age_group, 'Young Adult'):
        age = 20 * 12
    else:
        age = float('nan')

    # Extract sex. Use one-hot encoding.
    sex = get_sex(data)

    sex_features = np.zeros(2, dtype=int)
    if compare_strings(sex, 'Female'):
        sex_features[0] = 1
    elif compare_strings(sex, 'Male'):
        sex_features[1] = 1

    # Extract height and weight.
    height = get_height(data)
    weight = get_weight(data)

    # Extract pregnancy status.
    is_pregnant = get_pregnancy_status(data)

    # Extract recording locations and data. Identify when a location is present, and compute the mean, variance, and skewness of
    # each recording. If there are multiple recordings for one location, then extract features from the last recording.
    locations = get_locations(data)

    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    num_recording_locations = len(recording_locations)
    recording_features = np.zeros((num_recording_locations, 4), dtype=float)
    num_locations = len(locations)
    num_recordings = len(recordings)
    if num_locations==num_recordings:
        for i in range(num_locations):
            for j in range(num_recording_locations):
                if compare_strings(locations[i], recording_locations[j]) and np.size(recordings[i])>0:
                    recording_features[j, 0] = 1
                    recording_features[j, 1] = np.mean(recordings[i])
                    recording_features[j, 2] = np.var(recordings[i])
                    recording_features[j, 3] = sp.stats.skew(recordings[i])
    recording_features = recording_features.flatten()

    features = np.hstack(([age], sex_features, [height], [weight], [is_pregnant], recording_features))

    return np.asarray(features, dtype=np.float32)


In [95]:
murmur_classes

['Present', 'Unknown', 'Absent']


# evaluate 코드

In [41]:
#!/usr/bin/env python

# Do *not* edit this script. Changes will be discarded so that we can process the models consistently.

# This file contains functions for evaluating models for the 2022 Challenge. You can run it as follows:
#
#   python evaluate_model.py labels outputs scores.csv
#
# where 'labels' is a folder containing files with the labels, 'outputs' is a folder containing files with the outputs from your
# model, and 'scores.csv' (optional) is a collection of scores for the model outputs.
#
# Each label or output file must have the format described on the Challenge webpage. The scores for the algorithm outputs include
# the area under the receiver-operating characteristic curve (AUROC), the area under the recall-precision curve (AUPRC), macro
# accuracy, a weighted accuracy score, and the Challenge score.

import os, os.path, sys, numpy as np
from helper_code import load_patient_data, get_murmur, get_outcome, load_challenge_outputs, compare_strings

# Evaluate the models.
def evaluate_model(label_folder, output_folder):
    # Define murmur and outcome classes.
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']

    # Load and parse label and model output files.
    label_files, output_files = find_challenge_files(label_folder, output_folder)
    murmur_labels = load_murmurs(label_files, murmur_classes)
    murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
    outcome_labels = load_outcomes(label_files, outcome_classes)
    outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)

    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')

    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
        murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
        outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)

    # Return the results.
    return murmur_scores, outcome_scores

# Find Challenge files.
def find_challenge_files(label_folder, output_folder):
    label_files = list()
    output_files = list()
    for label_file in sorted(os.listdir(label_folder)):
        label_file_path = os.path.join(label_folder, label_file) # Full path for label file
        if os.path.isfile(label_file_path) and label_file.lower().endswith('.txt') and not label_file.lower().startswith('.'):
            root, ext = os.path.splitext(label_file)
            output_file = root + '.csv'
            output_file_path = os.path.join(output_folder, output_file) # Full path for corresponding output file
            if os.path.isfile(output_file_path):
                label_files.append(label_file_path)
                output_files.append(output_file_path)
            else:
                raise IOError('Output file {} not found for label file {}.'.format(output_file, label_file))

    if label_files and output_files:
        return label_files, output_files
    else:
        raise IOError('No label or output files found.')

# Load murmurs from label files.
def load_murmurs(label_files, classes):
    num_patients = len(label_files)
    num_classes = len(classes)

    # Use one-hot encoding for the labels.
    labels = np.zeros((num_patients, num_classes), dtype=np.bool_)

    # Iterate over the patients.
    for i in range(num_patients):
        data = load_patient_data(label_files[i])
        label = get_murmur(data)
        for j, x in enumerate(classes):
            if compare_strings(label, x):
                labels[i, j] = 1

    return labels

# Load outcomes from label files.
def load_outcomes(label_files, classes):
    num_patients = len(label_files)
    num_classes = len(classes)

    # Use one-hot encoding for the labels.
    labels = np.zeros((num_patients, num_classes), dtype=np.bool_)

    # Iterate over the patients.
    for i in range(num_patients):
        data = load_patient_data(label_files[i])
        label = get_outcome(data)
        for j, x in enumerate(classes):
            if compare_strings(label, x):
                labels[i, j] = 1

    return labels

# Load outputs from output files.
def load_classifier_outputs(output_files, classes):
    # The outputs should have the following form:
    #
    # #Record ID
    # class_1, class_2, class_3
    #       0,       1,       1
    #    0.12,    0.34,    0.56
    #
    num_patients = len(output_files)
    num_classes = len(classes)

    # Use one-hot encoding for the outputs.
    binary_outputs = np.zeros((num_patients, num_classes), dtype=np.bool_)
    scalar_outputs = np.zeros((num_patients, num_classes), dtype=np.float64)

    # Iterate over the patients.
    for i in range(num_patients):
        patient_id, patient_classes, patient_binary_outputs, patient_scalar_outputs = load_challenge_outputs(output_files[i])

        # Allow for unordered or reordered classes.
        for j, x in enumerate(classes):
            for k, y in enumerate(patient_classes):
                if compare_strings(x, y):
                    binary_outputs[i, j] = patient_binary_outputs[k]
                    scalar_outputs[i, j] = patient_scalar_outputs[k]

    return binary_outputs, scalar_outputs

# For each patient, set a specific class to positive if no class is positive or multiple classes are positive.
def enforce_positives(outputs, classes, positive_class):
    num_patients, num_classes = np.shape(outputs)
    j = classes.index(positive_class)

    for i in range(num_patients):
        if np.sum(outputs[i, :]) != 1:
            outputs[i, :] = 0
            outputs[i, j] = 1
    return outputs

# Compute macro AUROC and macro AUPRC.
def compute_auc(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    # Compute and summarize the confusion matrices for each class across at distinct output values.
    auroc = np.zeros(num_classes)
    auprc = np.zeros(num_classes)

    for k in range(num_classes):
        # We only need to compute TPs, FPs, FNs, and TNs at distinct output values.
        thresholds = np.unique(outputs[:, k])
        thresholds = np.append(thresholds, thresholds[-1]+1)
        thresholds = thresholds[::-1]
        num_thresholds = len(thresholds)

        # Initialize the TPs, FPs, FNs, and TNs.
        tp = np.zeros(num_thresholds)
        fp = np.zeros(num_thresholds)
        fn = np.zeros(num_thresholds)
        tn = np.zeros(num_thresholds)
        fn[0] = np.sum(labels[:, k] == 1)
        tn[0] = np.sum(labels[:, k] == 0)

        # Find the indices that result in sorted output values.
        idx = np.argsort(outputs[:, k])[::-1]

        # Compute the TPs, FPs, FNs, and TNs for class k across thresholds.
        i = 0
        for j in range(1, num_thresholds):
            # Initialize TPs, FPs, FNs, and TNs using values at previous threshold.
            tp[j] = tp[j-1]
            fp[j] = fp[j-1]
            fn[j] = fn[j-1]
            tn[j] = tn[j-1]

            # Update the TPs, FPs, FNs, and TNs at i-th output value.
            while i < num_patients and outputs[idx[i], k] >= thresholds[j]:
                if labels[idx[i], k]:
                    tp[j] += 1
                    fn[j] -= 1
                else:
                    fp[j] += 1
                    tn[j] -= 1
                i += 1

        # Summarize the TPs, FPs, FNs, and TNs for class k.
        tpr = np.zeros(num_thresholds)
        tnr = np.zeros(num_thresholds)
        ppv = np.zeros(num_thresholds)
        for j in range(num_thresholds):
            if tp[j] + fn[j]:
                tpr[j] = float(tp[j]) / float(tp[j] + fn[j])
            else:
                tpr[j] = float('nan')
            if fp[j] + tn[j]:
                tnr[j] = float(tn[j]) / float(fp[j] + tn[j])
            else:
                tnr[j] = float('nan')
            if tp[j] + fp[j]:
                ppv[j] = float(tp[j]) / float(tp[j] + fp[j])
            else:
                ppv[j] = float('nan')

        # Compute AUROC as the area under a piecewise linear function with TPR/
        # sensitivity (x-axis) and TNR/specificity (y-axis) and AUPRC as the area
        # under a piecewise constant with TPR/recall (x-axis) and PPV/precision
        # (y-axis) for class k.
        for j in range(num_thresholds-1):
            auroc[k] += 0.5 * (tpr[j+1] - tpr[j]) * (tnr[j+1] + tnr[j])
            auprc[k] += (tpr[j+1] - tpr[j]) * ppv[j+1]

    # Compute macro AUROC and macro AUPRC across classes.
    if np.any(np.isfinite(auroc)):
        macro_auroc = np.nanmean(auroc)
    else:
        macro_auroc = float('nan')
    if np.any(np.isfinite(auprc)):
        macro_auprc = np.nanmean(auprc)
    else:
        macro_auprc = float('nan')

    return macro_auroc, macro_auprc, auroc, auprc

# Compute a binary confusion matrix, where the columns are the expert labels and the rows are the classifier labels.
def compute_confusion_matrix(labels, outputs):
    assert(np.shape(labels)[0] == np.shape(outputs)[0])
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients = np.shape(labels)[0]
    num_label_classes = np.shape(labels)[1]
    num_output_classes = np.shape(outputs)[1]

    A = np.zeros((num_output_classes, num_label_classes))
    for k in range(num_patients):
        for i in range(num_output_classes):
            for j in range(num_label_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

# Compute binary one-vs-rest confusion matrices, where the columns are the expert labels and the rows are the classifier labels.
def compute_one_vs_rest_confusion_matrix(labels, outputs):
    assert(np.shape(labels) == np.shape(outputs))
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients, num_classes = np.shape(labels)

    A = np.zeros((num_classes, 2, 2))
    for i in range(num_patients):
        for j in range(num_classes):
            if labels[i, j] == 1 and outputs[i, j] == 1: # TP
                A[j, 0, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 1: # FP
                A[j, 0, 1] += 1
            elif labels[i, j] == 1 and outputs[i, j] == 0: # FN
                A[j, 1, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 0: # TN
                A[j, 1, 1] += 1

    return A

# Compute macro F-measure.
def compute_f_measure(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    A = compute_one_vs_rest_confusion_matrix(labels, outputs)

    f_measure = np.zeros(num_classes)
    for k in range(num_classes):
        tp, fp, fn, tn = A[k, 0, 0], A[k, 0, 1], A[k, 1, 0], A[k, 1, 1]
        if 2 * tp + fp + fn > 0:
            f_measure[k] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            f_measure[k] = float('nan')

    if np.any(np.isfinite(f_measure)):
        macro_f_measure = np.nanmean(f_measure)
    else:
        macro_f_measure = float('nan')

    return macro_f_measure, f_measure

# Compute accuracy.
def compute_accuracy(labels, outputs):
    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    num_patients, num_classes = np.shape(labels)
    A = compute_confusion_matrix(labels, outputs)

    # Compute accuracy.
    if np.sum(A) > 0:
        accuracy = np.trace(A) / np.sum(A)
    else:
        accuracy = float('nan')

    # Compute per-class accuracy.
    accuracy_classes = np.zeros(num_classes)
    for i in range(num_classes):
        if np.sum(A[:, i]) > 0:
            accuracy_classes[i] = A[i, i] / np.sum(A[:, i])
        else:
            accuracy_classes[i] = float('nan')

    return accuracy, accuracy_classes

# Compute accuracy.
def compute_weighted_accuracy(labels, outputs, classes):
    # Define constants.
    if classes == ['Present', 'Unknown', 'Absent']:
        weights = np.array([[5, 3, 1], [5, 3, 1], [5, 3, 1]])
    elif classes == ['Abnormal', 'Normal']:
        weights = np.array([[5, 1], [5, 1]])
    else:
        raise NotImplementedError('Weighted accuracy undefined for classes {}'.format(', '.join(classes)))

    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    A = compute_confusion_matrix(labels, outputs)

    # Multiply the confusion matrix by the weight matrix.
    assert(np.shape(A) == np.shape(weights))
    B = weights * A

    # Compute weighted_accuracy.
    if np.sum(B) > 0:
        weighted_accuracy = np.trace(B) / np.sum(B)
    else:
        weighted_accuracy = float('nan')

    return weighted_accuracy

# Define total cost for algorithmic prescreening of m patients.
def cost_algorithm(m):
    return 10*m

# Define total cost for expert screening of m patients out of a total of n total patients.
def cost_expert(m, n):
    return (25 + 397*(m/n) -1718*(m/n)**2 + 11296*(m/n)**4) * n

# Define total cost for treatment of m patients.
def cost_treatment(m):
    return 10000*m

# Define total cost for missed/late treatement of m patients.
def cost_error(m):
    return 50000*m

# Compute Challenge cost metric.
def compute_cost(labels, outputs, label_classes, output_classes):
    # Define positive and negative classes for referral and treatment.
    positive_classes = ['Present', 'Unknown', 'Abnormal']
    negative_classes = ['Absent', 'Normal']

    # Compute confusion matrix.
    A = compute_confusion_matrix(labels, outputs)

    # Identify positive and negative classes for referral.
    idx_label_positive = [i for i, x in enumerate(label_classes) if x in positive_classes]
    idx_label_negative = [i for i, x in enumerate(label_classes) if x in negative_classes]
    idx_output_positive = [i for i, x in enumerate(output_classes) if x in positive_classes]
    idx_output_negative = [i for i, x in enumerate(output_classes) if x in negative_classes]

    # Identify true positives, false positives, false negatives, and true negatives.
    tp = np.sum(A[np.ix_(idx_output_positive, idx_label_positive)])
    fp = np.sum(A[np.ix_(idx_output_positive, idx_label_negative)])
    fn = np.sum(A[np.ix_(idx_output_negative, idx_label_positive)])
    tn = np.sum(A[np.ix_(idx_output_negative, idx_label_negative)])
    total_patients = tp + fp + fn + tn

    # Compute total cost for all patients.
    total_cost = cost_algorithm(total_patients) \
        + cost_expert(tp + fp, total_patients) \
        + cost_treatment(tp) \
        + cost_error(fn)

    # Compute mean cost per patient.
    if total_patients > 0:
        mean_cost = total_cost / total_patients
    else:
        mean_cost = float('nan')

    return mean_cost



In [45]:
murmur_classes = ['Present', 'Unknown', 'Absent']

In [46]:
label_folder = "/home/hj20/hmd/murmur/test/"
output_folder ="./test_outputs"

In [47]:
label_files, output_files = find_challenge_files(label_folder, output_folder)

In [48]:
label_files[:5]

['/home/hj20/hmd/murmur/test/33151.txt',
 '/home/hj20/hmd/murmur/test/38337.txt',
 '/home/hj20/hmd/murmur/test/40798.txt',
 '/home/hj20/hmd/murmur/test/49558.txt',
 '/home/hj20/hmd/murmur/test/49572.txt']

In [49]:
output_files[:5]

['./test_outputs/33151.csv',
 './test_outputs/38337.csv',
 './test_outputs/40798.csv',
 './test_outputs/49558.csv',
 './test_outputs/49572.csv']

In [50]:
murmur_labels = load_murmurs(label_files, murmur_classes)

In [51]:
murmur_labels #해당클래스 양성 

array([[ True, False, False],
       [False, False,  True],
       [False, False,  True],
       [False,  True, False],
       [ True, False, False],
       [False,  True, False],
       [ True, False, False],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [ True, False, False],
       [False,  True, False],
       [ True, False, False],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [Fa

In [52]:
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)

In [53]:
murmur_binary_outputs

array([[False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [ True, False, False],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [Fa

In [54]:
murmur_labels

array([[ True, False, False],
       [False, False,  True],
       [False, False,  True],
       [False,  True, False],
       [ True, False, False],
       [False,  True, False],
       [ True, False, False],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [ True, False, False],
       [False,  True, False],
       [ True, False, False],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [Fa

In [168]:
 murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes)

In [169]:
 murmur_weighted_accuracy

0.3935309973045822

In [55]:
murmur_binary_outputs

array([[False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [ True, False, False],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [Fa

In [56]:
murmur_classes

['Present', 'Unknown', 'Absent']

In [ ]:
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)

    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')

In [ ]:
if __name__ == '__main__':
    murmur_scores, outcome_scores = evaluate_model(sys.argv[1], sys.argv[2])

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
        ','.join(classes),
        ','.join('{:.3f}'.format(x) for x in auroc_classes),
        ','.join('{:.3f}'.format(x) for x in auprc_classes),
        ','.join('{:.3f}'.format(x) for x in f_measure_classes),
        ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
        ','.join(classes),
        ','.join('{:.3f}'.format(x) for x in auroc_classes),
        ','.join('{:.3f}'.format(x) for x in auprc_classes),
        ','.join('{:.3f}'.format(x) for x in f_measure_classes),
        ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
        + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

    if len(sys.argv) == 3:
        print(output_string)
    elif len(sys.argv) == 4:
        with open(sys.argv[3], 'w') as f:
            f.write(output_string)
